In [1]:
## for colab
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
cp: cannot stat '/usr/lib/chromium-browser/chromedriver': No such file or directory


In [1]:
### for operabase
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import itertools
import datetime

###for google flights
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import html
import time
from selenium.webdriver.chrome.options import Options

In [2]:
#trainline
## trainline
import json
_DEFAULT_DATE_FORMAT=  '%Y-%m-%dT%H:%M:%S%z'
_SEARCH_URL = "https://www.trainline.eu/api/v5_1/search"

#4916;paris
#5892;london st-pancras
#5893;bruxelles-midi
#5894;amsterdam-centraal

passenger_list = None
arrival_station_id = 4916
departure_station_id = 5892
import datetime as dt

#date1 = dt.datetime.strptime("15/11/2019 08:00",)
search_date = dt.date.today()#"15/11/2019 08:00"
departure_date = search_date.strftime(_DEFAULT_DATE_FORMAT)



data = {
            "local_currency": "EUR",
            "search": {
                "passengers": passenger_list,
                "arrival_station_id": arrival_station_id,
                "departure_date": departure_date,
                "departure_station_id": departure_station_id,
                "systems": [
                    "sncf",
                    "db",
                    "idtgv",
                    "ouigo",
                    "trenitalia",
                    "ntv",
                    "hkx",
                    "renfe",
                    "cff",
                    "benerail",
                    "ocebo",
                    "westbahn",
                    "leoexpress",
                    "locomore",
                    "busbud",
                    "flixbus",
                    "distribusion",
                    "cityairporttrain",
                    "obb",
                    "timetable"
                ]
            }
        }
        
post_data = json.dumps(data)
c = requests.session()
ret = c.post(url=_SEARCH_URL, data=post_data)
print(ret)

#https://github.com/tducret/trainline-python/blob/master/trainline/__init__.py

<Response [415]>


# Classes

In [3]:
#needed in opera class
#opera_air_codes = pd.read_csv('opera_airport_codes.csv',header=0)
opera_air_codes = pd.read_csv('https://github.com/larkaa/opera_tools/raw/master/opera_airport_codes.csv',header=0)

class Opera:
    def __init__(self,title,code,comp,city,theater,country='',info='',airport = ''):
        self.title = title
        self.code = code
        self.composer = comp
        self.city = city
        self.theater = theater
        self.info = info
        self.country = country
        self.airport = airport       
        self._getAirport()
        
    def __str__(self):        
        return('{},{}| {}| {}| {}| {}'.format(self.city,
                                              self.country,
                                              self.title,
                                              self.theater,
                                              self.composer,
                                              self.code))  
    def _getAirport(self):
        global opera_air_codes
    
        a = opera_air_codes.Airport[opera_air_codes['City']==self.city]
        if a.values.size>0:
            self.airport = a.item()
            
    def getInfo(self):
        if self.info == '':
            url = 'https://www.operabase.com/ressources/views/festivals/season_prod_fetch.php?id={}&statut=&lang=en'.format(self.code)
            s = requests.session()
            try:
                res = s.get(url,timeout=3)
                
                temp = re.sub('href="(.*?)"', 'href="https://www.operabase.com\g<1>"',res.text)
                #res = cleanhtml(res.text)
                self.info = temp
            except Exception:
                self.info = Exception
                
        return self.info
    
         
class Weekend:
    def __init__(self,start,end,
                 operas=None,countries=None,airports=None,flights=None):
        self.start=start
        self.end=end
        if operas == None:
            self.operas = []
        else:
            self.operas=operas
        if countries == None:
            self.countries= []
        else:
            self.countries=countries
        if airports == None:
            self.airports= []
        else:
            self.airports=airports
        if flights == None:
            self.flights= []
        else:
            self.flights=flights
        
    def __getitem__(self, i):
        return self.operas[i]
    
    def _getCountries(self):
        temp = []
        for op in self.operas:
            temp.append(op.country)
            
        temp = set(temp)
        self.countries = temp

        
    def addOpera(self,opera):
        self.operas.append(opera)        
        self._getCountries()
        
    def addOperas(self,oplist):
        for op in oplist:
            self.operas.append(op)             
        self._getCountries()
                
    def getFlights(self,include_cities=None):
        heure_depart = "0800-2400" 
        heure_retour = "1200-2400"            

        to_find = []        
        if include_cities == None:
            for op in self.operas:
                to_find.append(op.airport)
        else:
            for op in self.operas:
                if op.city in include_cities:
                    to_find.append(op.airport)
        to_find = set(to_find)
        
        temp = []
        for dest in to_find:
            dest_ops = []
            dest_theaters = []

            for op in self.operas:
                if op.airport==dest:
                    dest_ops.append(op.title)
                    dest_theaters.append(op.theater)
                    
            dest_ops = set(dest_ops)   
            dest_theaters = set(dest_theaters)
            date_depart = '-'.join([self.start[0:4],self.start[4:6],self.start[6:]])
            date_retour = '-'.join([self.end[0:4],self.end[4:6],self.end[6:]])
            global opera_air_codes
            cities = opera_air_codes.City[opera_air_codes.Airport==dest].values
            #print('dest',dest)
            
            newflights = search_for_flights(dest, date_depart,heure_depart,  date_retour,heure_retour)                        
            
            temp_extract=[]
            for f in newflights:
                temp_extract = [dest_ops,dest_theaters,cities]
                temp_extract.extend(f.extract())

                temp.append(temp_extract)#3+12
            
            self.flights.extend(newflights)        
            
        #create df to save to csv
        df = pd.DataFrame(temp,columns=['Operas','Theater','City','Dates','Price',
                                        'Company Outbound',
                                        'Route Outbound','Time Outbound',
                                        'Duration Outbound','Direct Outbound?',
                                        'Company Inbound','Route Inbound',
                                        'Time Inbound','Duration Inbound',
                                        'Direct Inboud?']) # 15

        df.to_csv('flight_list.csv',index=False)
        print(df.values.tolist())
        return df.values.tolist()
            
        
    def print_flights(self):
        if self.flights==[]:
            return 0
        
        to_find = []        
        for op in self.operas:
            to_find.append(op.airport)
        temp = []        
        for dest in to_find:
            dest_ops = []
            dest_theaters = []

            for op in self.operas:
                if op.airport==dest:
                    dest_ops.append(op.title)
                    dest_theaters.append(op.theater)
                    
            dest_ops = set(dest_ops)   
            dest_theaters = set(dest_theaters)
            date_depart = '-'.join([self.start[0:4],self.start[4:6],self.start[6:]])
            date_retour = '-'.join([self.end[0:4],self.end[4:6],self.end[6:]])
            global opera_air_codes
            cities = opera_air_codes.City[opera_air_codes.Airport==dest].values.tolist()
            
            temp_extract = []
            for f in self.flights:
                temp_extract = [dest_ops,dest_theaters,cities]
                temp_extract.extend(f.extract())
                #print(temp_extract)

                temp.append(temp_extract)#3+12
            
            #self.flights.extend(f)  
        #create df to save to csv
        #print(temp)
        df = pd.DataFrame(temp,columns=['Operas','Theater','City','Dates','Price',
                                        'Company Outbound',
                                        'Route Outbound','Time Outbound',
                                        'Duration Outbound','Direct Outbound?',
                                        'Company Inbound','Route Inbound',
                                        'Time Inbound','Duration Inbound',
                                        'Direct Inboud?']) # 15

        #df.to_csv('flight_list.csv',index=False)
        return df.values.tolist()
        
        
    def show(self):
        print('Period of {} - {}'.format(self.start,self.end))
        for c in self.countries:
            print('\t{}:'.format(c))
               
            for op in self.operas:
                if op.country == c:
                    print('\t',op)
                    
    def get_table(self):
        res = []
        #res = ['Weekend of {}-{}'.format(self.start,self.end)]
        for c in self.countries:
            for i,op in enumerate(self.operas):
                if op.country == c:
                    res.append(['{}-{}'.format(self.start,self.end),op.country,op.city,op.theater,op.title,op.composer,op.code,i])
        return res

In [4]:
class Flight:
    def __init__(self,dates,price,company1,codes1,time1,duration1,direct1,
                    company2, codes2, time2, duration2, direct2):
                 
        self.dates = dates
        self.price = price
        self.company1 = company1
        self.codes1 = codes1
        self.time1 = time1
        self.duration1 = duration1
        self.direct1 = direct1
        self.company2 = company2
        self.codes2 = codes2
        self.time2 = time2
        self.duration2 = duration2
        self.direct2 = direct2
             
    def extract(self):
        return([
            self.dates,
            self.price,
            self.company1,
            self.codes1,
            self.time1,
            self.duration1,
            self.direct1,
            self.company2,
            self.codes2,
            self.time2,
            self.duration2,
            self.direct2])

        
class Weekday:
    def __init__(self,start,end,airport, flights=None, operas=None):
        self.start = start
        self.end = end
        self.airport = airport
        if flights==None:
            self.flights = []
        else:
            self.flights = flights
        if operas==None:
            self.operas = []
        else:
            self.operas = operas
            
    def extract(self):
        return 0
        
    def printWeekday(self):
        print(self.airport, 'from ',self.start,' to ', self.end)
        print('Operas')
        for o in self.operas:
            print('\t',o.title, o.composer, o.getInfo())
        print('Flights')
        for f in self.flights:
            print('\tOutbound:',f.price, f.company1, f.time1, f.duration1,
                  '\tInbound:', f.time2, f.duration2)


## Operabase code

Sune Hjerrild, CEO | 
Mike Gibb, Creative Director | 
Muriel Denzler, COO | 
Armand Arapian, Director of Programming

In [5]:
def cleanhtml(raw_html):
    #cleanr = re.compile('<.*?>')
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return cleantext
    
## get results of weekends,
## organize in weekend dates, cities 
##input start date, end date, and countries you want to visit
def get_operas_in_period(start,finish,countries):
    weekends = get_weekends(start,finish)
    
    s = requests.session()
    #iso 
    #"w" - western europe
    #"e" - eastern europe
    #"wd" - world
    #"s" - scandinavia
    url = 'https://www.operabase.com/en'   
    
    for weekend in weekends:
        data = {"country_iso": "wd","from":weekend.start,"to":weekend.end}
        r = s.post(url, data = data)
        #r = get_search_results(start,finish, countries=countries)
        perfs =  get_performances(r.text,countries)
        #print(perfs)
        weekend.addOperas(perfs)
        #res.append(weekend)
        
    return weekends

def get_operas_in_dates(start,finish,countries):
    s = requests.session()
    #iso 
    #"w" - western europe
    #"e" - eastern europe
    #"wd" - world
    #"s" - scandinavia
    url = 'https://www.operabase.com/en' 
    date_start = datetime.datetime.strptime(start, '%Y%m%d')
    date_finish = datetime.datetime.strptime(finish, '%Y%m%d')
    
    weekend = Weekend(date_start.strftime("%Y%m%d"),date_finish.strftime("%Y%m%d"))
    
    
    data = {"country_iso": "wd","from":weekend.start,"to":weekend.end}
    r = s.post(url, data = data)
    #r = get_search_results(start,finish, countries=countries)
    perfs =  get_performances(r.text,countries)
    #print(perfs)
    weekend.addOperas(perfs)
    #res.append(weekend)
        
    return weekend
    
    
    
#only works for weekend searches. 
# for long-term seraches, the output is different
def get_performances(html,country_filter):
    # split by country
    countries = split_countries(html)
    
    opera_list = []
    for name,text in countries:
        #print(name)
        if name in country_filter:
            temp = get_operas(text,name)
            opera_list.extend(temp)

    return(opera_list)
    
    
def get_operas(html,cname):
    # get composer, city & venue
    temp = re.findall('\|</span>(.*?)<',html)
    ccv = [temp[i:i+3] for i in range(0,len(temp),3)]
    
    #get performances & id codes
    parsed_html = BeautifulSoup(html)
    operas_temp = parsed_html.body.find_all('span', onclick=True)
    temp = []
    for a in operas_temp:
        b = re.findall("perdate\('(.*?)'.*?>(.*?)<", str(a))
        temp.append(list(b[0]))
        
    #print(temp)
    res = []
    for i,row in enumerate(ccv):
        #title,code,comp,city,theater
        res.append([temp[i][1],temp[i][0],row[0],row[1],row[2]])
    
    res.sort()
    res2 = list(k for k,_ in itertools.groupby(res))
    #print(res2)
    
    res = []
    for i in res2:
        res.append(Opera(*i,country=cname))
        
    return res
    
def split_countries(html):
    pattern = '</div></div><div class="list_title_medium" >(.*?)<'
    string = html

    #to find indices of all matches
    splits = [(m.start(0), m.end(0)) for m in re.finditer(pattern,string)]
    #to find country names
    countries = [re.findall('list_title_medium" >(.*?)<',string[a:b])[0].strip() for a,b in splits] 
    
    res = []
    #match country names and corresponding text
    for i in range(len(splits)):
        if i < len(splits)-1:
            text = string[splits[i][1]:splits[i+1][0]]
            res.append([countries[i],text])
        else:
            text = string[splits[-1][1]:]
            res.append([countries[-1],text])
        
    return res


def get_weekends(start,finish):
    res = []    
    
    date_start = datetime.datetime.strptime(start, '%Y%m%d')
    date_finish = datetime.datetime.strptime(finish, '%Y%m%d')
    
    while date_start <= date_finish:
        
        weekno = date_start.weekday()
        #print(weekno)
        if weekno>=4:
            temp = date_start
            
            while True:
                weekno = (temp + datetime.timedelta(days=1)).weekday()
                if weekno<4:
                    #print([date_start,temp])
                    res.append(Weekend(date_start.strftime("%Y%m%d"),temp.strftime("%Y%m%d")))
                    date_start = temp + datetime.timedelta(days=1)
                    break
                 
                temp = temp+datetime.timedelta(days=1)
        else:   
            date_start = date_start + datetime.timedelta(days=1)
            
    return res 


## Google flight code

In [6]:
### search just for a weekend in a city
### then show if any operas are going on.

def get_operas_in_city(start,end,airports):
    s = requests.session()
    #iso 
    #"w" - western europe
    #"e" - eastern europe
    #"wd" - world
    #"s" - scandinavia
    url = 'https://www.operabase.com/en'       
    
    data = {"country_iso": "wd","from":start,"to":end}
    r = s.post(url, data = data)
    #print(r.text)
    
    global opera_air_codes
        
    countries = []
    for a in airports:
        temp = opera_air_codes.Country[opera_air_codes.Airport==a].values.tolist()
        countries.extend(temp)
    #print(countries)
    countries = list(set(countries))
    operas =  get_performances(r.text,countries)
    res = []
    
    for o in operas:
        #print(o)
        if o.airport in airports:
            res.append(o)
            
    return res

def flight_search_dates(countries,start,end,departure_time="0800-2400",return_time="1000-2400"):
    global opera_air_codes
    airports = []
    
    for c in countries:
        temp = opera_air_codes.Airport[opera_air_codes.Country==c].values.tolist()
        airports.extend(temp)
    airports = list(set(airports))
    #print(airports)
    
    
    # 1 setup a weekend
    we = get_operas_in_dates(start,end,countries)
    
    # 2 get flights for that weekend
    df = we.getFlights()
    
    # 3 return df and weekend
    return(we,df)
    
    '''
    operas = get_operas_in_city(start,end,airports)        
    res = []
    
    date_depart = '-'.join([start[0:4],start[4:6],start[6:]])
    date_retour = '-'.join([end[0:4],end[4:6],end[6:]])
        
    for a in airports:        
        flights = search_for_flights(a,date_depart,departure_time,date_retour,return_time)
        temp = []
        for o in operas:
            if o.airport == a:
                temp.append(o)
        country = opera_air_codes.Country[opera_air_codes.Airport==a].values.tolist()
        
        #print(start,end,operas,country,a)
        
        res.append(Weekend(start,end,operas,country,a,flights))
        
        #res.append(Weekday(start,end,a,flights,temp))

    return res'''
    
#wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
#wd.get("https://www.webite-url.com")

aero_depart = '/m/05qtj' #PAR
aero_arrivee = 'DUB'
date_depart = "2019-08-23"
heure_depart = "1800-2400"
date_retour = "2019-08-25"
heure_retour = "1200-2400"
time_sleep = 1

def search_for_flights(aero_arrivee,date_depart,heure_depart,date_retour,heure_retour,time_sleep=2,aero_depart = '/m/05qtj'):
    
    url = 'https://www.google.com/flights/#flt=' + aero_depart +"."+ aero_arrivee + "." + date_depart+"*"+aero_arrivee+"."   +aero_depart +"." + date_retour  +";c:EUR;e:1;s:0*0;dt:"+heure_depart+";at:"+heure_retour+";sd:1;t:f"

    chrome_options = Options()  
    chrome_options.add_argument("--headless")
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    ##for colab
    #browser = webdriver.Chrome('chromedriver', options=chrome_options)
    browser =  webdriver.Chrome(executable_path='/usr/bin/chromedriver', options=chrome_options)
    #browser =  webdriver.Chrome(executable_path='C:\\Users\\Drew\\code\\opera_tools\\chromedriver' , options=chrome_options)
   
    #driver url 'https://chromedriver.storage.googleapis.com/76.0.3809.68/chromedriver_linux64.zip'

    browser.get(url)
    time.sleep(time_sleep)
    #source = browser.page_source

    
    num_aller = len(browser.find_elements_by_xpath('//li[contains(@class,"gws-flights-results__result-item")]'))
    #print(num_aller)
    res = []
    if num_aller>5:
        num_aller = num_aller-1
    for i in range(num_aller):
        clicks = browser.find_elements_by_xpath('//li[contains(@class,"gws-flights-results__result-item")]')
        #print('num_aller {} i {} len(click {})'.format(num_aller, i, len(clicks)))
        #print(i,'\n',clicks)
        aller = (clicks[i].text)
        price,company,codes,time1,duration,direct = split_flight_info(aller)
        
        #print('ALLER\n',clicks[i].text)
        #print()
        clicks[i].click()
 
        time.sleep(time_sleep)
        a = browser.find_elements_by_xpath('//li[contains(@class,"gws-flights-results__result-item")]')
        if len(a)>2:
            a = a[0:2]
        for j in a:
            price,company2,codes2,time2,duration2,direct2 =  split_flight_info(j.text)
            res.append(Flight(' - '.join([date_depart,date_retour]),
                              price, company, codes, time1, duration, direct,
                             company2, codes2, time2, duration2, direct2))
        
        #retour= [i.text for i in a]
        #a_r.append([aller,retour])
   
        #print('RETOUR\n',retour)
        browser.execute_script("window.history.go(-1)")
        #print(time)
        time.sleep(time_sleep)
        #print()
    browser.quit()  
    return res
    
    
# alternate way to get flight info
#parsed_html = BeautifulSoup(source)
#vols = parsed_html.body.findAll('jsl', attrs={'jstcache':'9041'})
def split_flight_info(text):
    a = text.split('\n')
    #['6:30 PM – 7:55 PM', 'Vueling', 'Iberia, British Airways', '1h 25m', 'ORY–MXP', 'Nonstop', '€105', 'round trip']
    #print(a)
    if len(a)== 7:
        time,company,duration,codes,direct,price,_ = a
    elif len(a)==8:
        time,_,_,duration,codes,direct,price,_ = a
        company = a[1]+'\n'+a[2]
    else:
        print("error")
        
    return(price,company,codes,time,duration,direct)    

## Testing

In [23]:
#20191108 - 20191110
test,df = flight_search_dates(['Norway'],'20191111','20191123')

test.show()
test.print_flights()
#for i in weekday:
#    #print(i)
#    i.show()
#    i.print_flights()

[[{'Peter Grimes'}, {'Grieghallen'}, array(['Bergen'], dtype=object), '2019-11-11 - 2019-11-23', '109 €', 'Air France', 'CDG–BGO', '14:30 – 16:45', '2 h 15 min', 'Sans escale', 'Air France', 'BGO–CDG', '17:35 – 20:00', '2 h 25 min', 'Sans escale'], [{'Hans og Grete ', 'Peter Grimes'}, {'Den Norske Opera'}, array(['Oslo'], dtype=object), '2019-11-11 - 2019-11-23', '99 €', 'Air France', 'CDG–OSL', '20:45 – 23:10', '2 h 25 min', 'Sans escale', 'Air France', 'OSL–CDG', '06:05 – 08:50', '2 h 45 min', 'Sans escale'], [{'Hans og Grete ', 'Peter Grimes'}, {'Den Norske Opera'}, array(['Oslo'], dtype=object), '2019-11-11 - 2019-11-23', '99 €', 'Air France', 'CDG–OSL', '20:45 – 23:10', '2 h 25 min', 'Sans escale', 'Air France', 'OSL–CDG', '12:25 – 14:55', '2 h 30 min', 'Sans escale']]
Period of 20191111 - 20191123
	Norway:
	 Oslo,Norway| Hans og Grete | Den Norske Opera| Humperdinck| 120002
	 Oslo,Norway| Peter Grimes| Den Norske Opera| Britten| 120987
	 Bergen,Norway| Peter Grimes| Grieghallen| 

,Operas,Theater,City,Dates,Price,Company Outbound,Route Outbound,Time Outbound,Duration Outbound,Direct Outbound?,Company Inbound,Route Inbound,Time Inbound,Duration Inbound,Direct Inboud?
0,"{Hans og Grete , Peter Grimes}",{Den Norske Opera},[Oslo],2019-11-11 - 2019-11-23,109 €,Air France,CDG–BGO,14:30 – 16:45,2 h 15 min,Sans escale,Air France,BGO–CDG,17:35 – 20:00,2 h 25 min,Sans escale
1,"{Hans og Grete , Peter Grimes}",{Den Norske Opera},[Oslo],2019-11-11 - 2019-11-23,99 €,Air France,CDG–OSL,20:45 – 23:10,2 h 25 min,Sans escale,Air France,OSL–CDG,06:05 – 08:50,2 h 45 min,Sans escale
2,"{Hans og Grete , Peter Grimes}",{Den Norske Opera},[Oslo],2019-11-11 - 2019-11-23,99 €,Air France,CDG–OSL,20:45 – 23:10,2 h 25 min,Sans escale,Air France,OSL–CDG,12:25 – 14:55,2 h 30 min,Sans escale
3,"{Hans og Grete , Peter Grimes}",{Den Norske Opera},[Oslo],2019-11-11 - 2019-11-23,109 €,Air France,CDG–BGO,14:30 – 16:45,2 h 15 min,Sans escale,Air France,BGO–CDG,17:35 – 20:00,2 h 25 min,Sans escale
4,"{Hans og Grete , Peter Grimes}",{Den Norske Opera},[Oslo],2019-11-11 - 2019-11-23,99 €,Air France,CDG–OSL,20:45 – 23:10,2 h 25 min,Sans escale,Air France,OSL–CDG,06:05 – 08:50,2 h 45 min,Sans escale
5,"{Hans og Grete , Peter Grimes}",{Den Norske Opera},[Oslo],2019-11-11 - 2019-11-23,99 €,Air France,CDG–OSL,20:45 – 23:10,2 h 25 min,Sans escale,Air France,OSL–CDG,12:25 – 14:55,2 h 30 min,Sans escale
6,{Peter Grimes},{Grieghallen},[Bergen],2019-11-11 - 2019-11-23,109 €,Air France,CDG–BGO,14:30 – 16:45,2 h 15 min,Sans escale,Air France,BGO–CDG,17:35 – 20:00,2 h 25 min,Sans escale
7,{Peter Grimes},{Grieghallen},[Bergen],2019-11-11 - 2019-11-23,99 €,Air France,CDG–OSL,20:45 – 23:10,2 h 25 min,Sans escale,Air France,OSL–CDG,06:05 – 08:50,2 h 45 min,Sans escale
8,{Peter Grimes},{Grieghallen},[Bergen],2019-11-11 - 2019-11-23,99 €,Air France,CDG–OSL,20:45 – 23:10,2 h 25 min,Sans escale,Air France,OSL–CDG,12:25 – 14:55,2 h 30 min,Sans escale
9,"{Hans og Grete , Peter Grimes}",{Den Norske Opera},[Oslo],2019-11-11 - 2019-11-23,109 €,Air France,CDG–BGO,14:30 – 16:45,2 h 15 min,Sans escale,Air France,BGO–CDG,17:35 – 20:00,2 h 25 min,Sans escale


In [19]:
start = '20191107'
stop = '20191203'
countries = ['Denmark','Sweden','Norway','Belgium','Portugal',
             'Finland','Austria', 'United Kingdom']


weekends = get_operas_in_period(start,stop,countries)
#test = get_performances(start,stop,countries)
temp = weekends[1].get_table()
print(temp)
for i in weekends:
    i.show()

['Weekend of 20191115 - 20191117', ['Sweden', 'Karlstad', 'Stora Scenen', 'Albert Herring', 'Britten', '121992'], ['Sweden', 'Göteborg', 'Operan', 'Carmen', 'Bizet', '118995'], ['Sweden', 'Malmö', 'Malmö Opera', 'Requiem', 'Mozart', '119906'], ['Sweden', 'Stockholm', 'Folkoperan', 'Tolvskillingsoperan ', 'Weill', '118753'], ['Sweden', 'Stockholm', 'Kungliga Operan', 'Trollflöjten ', 'Mozart', '118736'], ['Finland', 'Helsinki', 'Opera House', 'Höstsonaten', 'Fagerlund', '118733'], ['Austria', 'Wien', 'Staatsoper', 'Ariodante', 'Handel', '119103'], ['Austria', 'Wien', 'Volksoper', 'Das Gespenst von Canterville', 'Lange', '120572'], ['Austria', 'Wien', 'Halle E im MuseumsQuartier', 'Der Reigen ', 'Lang,B', '122738'], ['Austria', 'Waldkraiburg', 'Haus der Kultur', 'Der Vetter aus Dingsda', 'Künneke', '117040'], ['Austria', 'Wien', 'Volksoper', 'Die Csárdásfürstin', 'Kálmán,E', '120584'], ['Austria', 'Linz', 'Musiktheater', 'Die Entführung aus dem Serail', 'Mozart', '119618'], ['Austria', '

In [60]:
for i in test:
    i.show()
#print(test)

Weekend of 20190920 - 20190922
	Italy:
	 Spoleto,Italy| Il Barbiere di Siviglia| Teatro Nuovo Gian Carlo Menotti| Rossini| 124191
	 Venezia,Italy| Il barbiere di Siviglia| Teatro La Fenice| Rossini| 112018
	 Bari,Italy| Il barbiere di Siviglia| Teatro Petruzzelli| Rossini| 114378
	 Palermo,Italy| Il barbiere di Siviglia| Teatro Massimo| Rossini| 114480
	 Milano,Italy| L'elisir d'amore| Teatro alla Scala| Donizetti| 111195
	 Panicale,Italy| L'inganno felice| Teatro Cesare Caporali| Rossini| 119937
	 Palermo,Italy| La Traviata| Teatro Massimo| Verdi| 114479
	 Ancona,Italy| La Traviata| Teatro delle Muse| Verdi| 122541
	 Venezia,Italy| La scala di seta| Teatro La Fenice| Rossini| 112022
	 Firenze,Italy| La traviata| Teatro del Maggio Musicale Fiorentino| Verdi| 111702
	 Napoli,Italy| La traviata| Teatro di San Carlo| Verdi| 112101
	 Venezia,Italy| Luci mie traditrici| Teatro Malibran| Sciarrino| 112021
	 Venezia,Italy| Madama Butterfly| Teatro La Fenice| Puccini| 112020
	 Firenze,Italy| N

In [56]:
test[0].getFlights(include_cities=['Firenze'])


['2019-10-04 - 2019-10-06', '143 €', 'Air France', 'CDG–ARN', '20:50 – 23:25', '2 h 35 min', 'Sans escale', 'Air France', 'ARN–CDG', '06:00 – 08:45', '2 h 45 min', 'Sans escale']
[{'Tolvskillingsoperan ', 'Der ferne Klang'}, ['Kungliga Operan', 'Folkoperan'], array(['Stockholm'], dtype=object), '2019-10-04 - 2019-10-06', '143 €', 'Air France', 'CDG–ARN', '20:50 – 23:25', '2 h 35 min', 'Sans escale', 'Air France', 'ARN–CDG', '06:00 – 08:45', '2 h 45 min', 'Sans escale']
['2019-10-04 - 2019-10-06', '143 €', 'Air France', 'CDG–ARN', '20:50 – 23:25', '2 h 35 min', 'Sans escale', 'Air France', 'ARN–CDG', '10:40 – 13:20', '2 h 40 min', 'Sans escale']
[{'Tolvskillingsoperan ', 'Der ferne Klang'}, ['Kungliga Operan', 'Folkoperan'], array(['Stockholm'], dtype=object), '2019-10-04 - 2019-10-06', '143 €', 'Air France', 'CDG–ARN', '20:50 – 23:25', '2 h 35 min', 'Sans escale', 'Air France', 'ARN–CDG', '10:40 – 13:20', '2 h 40 min', 'Sans escale']
['2019-10-04 - 2019-10-06', '116 €', 'Norwegian Air 

,Operas,Theater,City,Dates,Price,Company Outbound,Route Outbound,Time Outbound,Duration Outbound,Direct Outbound?,Company Inbound,Route Inbound,Time Inbound,Duration Inbound,Direct Inboud?
0,"{Tolvskillingsoperan , Der ferne Klang}","[Kungliga Operan, Folkoperan]",[Stockholm],2019-10-04 - 2019-10-06,143 €,Air France,CDG–ARN,20:50 – 23:25,2 h 35 min,Sans escale,Air France,ARN–CDG,06:00 – 08:45,2 h 45 min,Sans escale
1,"{Tolvskillingsoperan , Der ferne Klang}","[Kungliga Operan, Folkoperan]",[Stockholm],2019-10-04 - 2019-10-06,143 €,Air France,CDG–ARN,20:50 – 23:25,2 h 35 min,Sans escale,Air France,ARN–CDG,10:40 – 13:20,2 h 40 min,Sans escale
2,{Carmen},[Det Kongelige Teater],"[København, Malmö]",2019-10-04 - 2019-10-06,116 €,Norwegian Air International,ORY–CPH,21:45 – 23:40,1 h 55 min,Sans escale,Norwegian Air International,CPH–ORY,19:20 – 21:20,2 h 0 min,Sans escale
3,{Carmen},[Det Kongelige Teater],"[København, Malmö]",2019-10-04 - 2019-10-06,118 €,SAS,CDG–CPH,19:45 – 21:30,1 h 45 min,Sans escale,SAS,CPH–CDG,17:15 – 19:05,1 h 50 min,Sans escale
4,{Carmen},[Det Kongelige Teater],"[København, Malmö]",2019-10-04 - 2019-10-06,118 €,SAS,CDG–CPH,19:45 – 21:30,1 h 45 min,Sans escale,SAS,CPH–CDG,19:40 – 21:30,1 h 50 min,Sans escale
5,{Carmen},[Det Kongelige Teater],"[København, Malmö]",2019-10-04 - 2019-10-06,123 €,Air France,CDG–CPH,21:00 – 22:55,1 h 55 min,Sans escale,Air France,CPH–CDG,06:15 – 08:15,2 h 0 min,Sans escale
6,{Carmen},[Det Kongelige Teater],"[København, Malmö]",2019-10-04 - 2019-10-06,197 €,Air France,CDG–CPH,21:00 – 22:55,1 h 55 min,Sans escale,Air France,CPH–CDG,15:05 – 17:05,2 h 0 min,Sans escale
7,{Rigoletto},[Den Norske Opera],[Oslo],2019-10-04 - 2019-10-06,143 €,Air France,CDG–OSL,20:50 – 23:10,2 h 20 min,Sans escale,Air France,OSL–CDG,06:15 – 08:45,2 h 30 min,Sans escale
8,{Rigoletto},[Den Norske Opera],[Oslo],2019-10-04 - 2019-10-06,159 €,Air France,CDG–OSL,20:50 – 23:10,2 h 20 min,Sans escale,Air France,OSL–CDG,12:30 – 14:55,2 h 25 min,Sans escale


## GUI testing

In [7]:
## dynamic adding of check boxes
from PyQt5 import QtWidgets, QtGui, QtCore
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import (QApplication, QCheckBox, QComboBox, QDateTimeEdit,
        QDial, QDialog, QGridLayout, QGroupBox, QHBoxLayout, QLabel, QLineEdit,
        QProgressBar, QPushButton, QRadioButton, QScrollBar, QSizePolicy,
        QSlider, QSpinBox, QStyleFactory, QTableWidget, QTableView,QTabWidget, QTextEdit,QTableWidgetItem,
        QVBoxLayout, QWidget)
import sys


class Window(QtWidgets.QWidget):
    def __init__(self, parent=None):
        
        super(Window, self).__init__(parent)
                
        self.mainLayout = QGroupBox(self)
        self.mainLayout.setTitle("Countries")
        grid1 = QGridLayout()
        grid2 = QGridLayout()
        hbox = QHBoxLayout()
        hbox2 = QHBoxLayout()
        vbox = QVBoxLayout()      
        vbox2 = QVBoxLayout() 
        self.weekends = []
        
        searchWeekends = QPushButton("Opera Weekends")
        searchWeekends.setDefault(False)
        searchWeekends.clicked.connect(self.get_opera_weekends)
        
        
        search_dates = QPushButton("Specific Dates")
        search_dates.setDefault(False)
        search_dates.clicked.connect(self.get_specific_dates)
        
        
        btn_search_flight = QPushButton("Search Flights")
        btn_search_flight.setDefault(False)
        btn_search_flight.clicked.connect(self.get_flights)
        
        btn_opera_info = QPushButton("Get Opera Info")
        btn_opera_info.setDefault(False)
        btn_opera_info.clicked.connect(self.get_opera_info)
        
        now = QtCore.QDate.currentDate()
        self.dateTimeEdit1 = QDateTimeEdit(self)
        self.dateTimeEdit1.setDate(now)
        self.dateTimeEdit1.setDisplayFormat("yyyy.MM.dd")
        
        self.dateTimeEdit2 = QDateTimeEdit(self)
        self.dateTimeEdit2.setDate(now.addMonths(1))
        self.dateTimeEdit2.setDisplayFormat("yyyy.MM.dd")       

        self.weekends = []
        self.checkboxes = []
        self.c_list = ['Czech Republic','Denmark','Estonia','Finland',
         'France','Germany','Greece','Hungary','Iceland',
         'Ireland','Italy','Latvia','Lithuania','Netherlands',
         'Norway','Poland','Portugal','Romania','Slovakia',
         'Slovenia','Spain','Sweden','Ukraine','United Kingdom' ]
        
        self.c_checked = [False,True,False,True,
         True,True,False,True,False,
         False,False,False,False,False,
         True,False,False,False,False,
         False,False,True,False,True ]       
    
        grid2.addWidget(searchWeekends,0,0,1,2) 
        grid2.addWidget(search_dates,0,3,1,2)
        grid2.addWidget(self.dateTimeEdit1,1,0,1,2)
        grid2.addWidget(self.dateTimeEdit2,1,3,1,2)
        grid2.setRowStretch(2,5)        
        
        for i,c in enumerate(self.c_list):
            self.addCheckbox(c)
            self.checkboxes[i].setChecked(self.c_checked[i])
            temp = 12
            if i<temp:
                grid1.addWidget(self.checkboxes[i],i+2,0,1,1)
            else:
                grid1.addWidget(self.checkboxes[i],i-temp+2,1,1,1)

        
        #table
        #self.tableWidget = QtWidgets.QTableWidget(self)
        #self.tableWidget.setObjectName("tableView")
        #self.rightTabWidget = QTabWidget()
        self.widget1 = QtWidgets.QWidget()
        self.widget1.setGeometry(QtCore.QRect(20, 10, 800, 350))
        self.widget1.setObjectName("widget1")
        
        self.tabWidget = QtWidgets.QTabWidget(self.widget1)
        self.tabWidget.setObjectName("tabWidget")
        self.Operas = QtWidgets.QWidget()
        self.Operas.setObjectName("Operas")
        self.table_operas = QtWidgets.QTableWidget(self.Operas)
        #self.table_operas.setSizeAdjustPolicy(QtWidgets.QAbstractScrollArea.AdjustToContents)
        
        table_dims = QtCore.QRect(10, 10, 830, 340)
        
        self.table_operas.setGeometry(table_dims)
        self.table_operas.setObjectName("table_operas")
        self.table_operas.setColumnCount(0)
        self.table_operas.setRowCount(0)
        self.tabWidget.addTab(self.Operas, "Operas")
        self.Flights = QtWidgets.QWidget()
        self.Flights.setObjectName("Flights")
        self.table_flights = QtWidgets.QTableWidget(self.Flights)
        self.table_flights.setGeometry(table_dims)
        self.table_flights.setObjectName("table_flights")
        self.table_flights.setColumnCount(0)
        self.table_flights.setRowCount(0)
        self.tabWidget.addTab(self.Flights, "Flights")
        #self.tabWidget.addTab(self.Flights, "")
        #self.verticalLayout.addWidget(self.tabWidget)
        
        #self.createTable()
        

        vbox.addLayout(grid2)
        vbox.addLayout(grid1)
        hbox.addLayout(vbox)
        hbox2.addWidget(btn_search_flight)
        hbox2.addWidget(btn_opera_info)
        
        
        #vbox2.addWidget(btn_search_flight)
        #vbox2.addWidget(btn_opera_info)
        vbox2.addLayout(hbox2)
        vbox2.addWidget(self.tabWidget)
        
        
        
        hbox.addLayout(vbox2)
        
        self.mainLayout.setLayout(hbox) 
        #self.mainLayout.setLayout(vbox)
        self.mainLayout.setGeometry(10,20,1200,500)
        self.mainLayout.setSizePolicy(QSizePolicy.Ignored, QSizePolicy.Ignored)
        self.show()
        
    def createBottomLeftTabWidget(self,weekends):
        self.bottomLeftTabWidget = QTabWidget()
        self.bottomLeftTabWidget.setSizePolicy(QSizePolicy.Preferred,
                QSizePolicy.Ignored)

        #weekend = weekends
        #title = weekend[0]
        #table_input = weekend[1]
        y_dim = len(table_input)
        x_dim = 5 + 1
        
        tab1 = QWidget()
        tableWidget = QTableWidget(x_dim, y_dim)

        tab1hbox = QHBoxLayout()
        tab1hbox.setContentsMargins(5, 5, 5, 5)
        tab1hbox.addWidget(tableWidget)
        tab1.setLayout(tab1hbox)

        tab2 = QWidget()
        textEdit = QTextEdit()

        textEdit.setPlainText("Twinkle, twinkle, little star,\n"
                              "How I wonder what you are.\n" 
                              "Up above the world so high,\n"
                              "Like a diamond in the sky.\n"
                              "Twinkle, twinkle, little star,\n" 
                              "How I wonder what you are!\n")

        tab2hbox = QHBoxLayout()
        tab2hbox.setContentsMargins(5, 5, 5, 5)
        tab2hbox.addWidget(textEdit)
        tab2.setLayout(tab2hbox)

        self.bottomLeftTabWidget.addTab(tab1, "&Table")
        self.bottomLeftTabWidget.addTab(tab2, "Text &Edit")
        
        
    
        
    def get_flights(self):

        we_index = []
        include_cities = []
        #weekend_index: row 8 of opera_list is the weekend index
        for index in self.table_operas.selectedIndexes():
            we_index.append(self.opera_list[index.row()][8])
            include_cities.append(self.opera_list[index.row()][2])
        
        we_index = list(set(we_index))
        print(we_index)

        table_input = []
        for w in we_index:
            i = self.weekends[w]        
            table_input.extend(i.getFlights(include_cities))

        print(table_input)
        self.flight_list = table_input
        
        self.update_flight_table()
        
        
        def update_flight_table(self):
            
            table_input = self.table_input

        
            rows = len(table_input)
            cols = 11
        
#['Operas','Theater','City','Dates','Price',
#                                        'Company Outbound',
#                                        'Route Outbound','Time Outbound',
#                                        'Duration Outbound','Direct Outbound?',
#                                        'Company Inbound','Route Inbound',
##                                        'Time Inbound','Duration Inbound',
#                                        'Direct Inboud?']
            myorder = [3,2,4,0,5,6,7,8,11,12,13]

            self.table_operas.setFont(QtGui.QFont('Arial', 8))
            #self.table_flights.setFont(QtGui.QFont('Arial', 7))
            self.table_flights.setRowCount(rows)
            self.table_flights.setColumnCount(cols)
            self.table_flights.setHorizontalHeaderLabels(['Dates', 'City','Price',"Title",
                                                          'Company','Route','Time','Duration',
                                                          'Route','Time','Duration'])
            #cols 6,7,8 = code, opera_index, weekend_index
        
            for i in range(rows): 
                for j in range(cols):
                    #print(table_input[i][j])
                    self.table_flights.setItem(i,j,QTableWidgetItem(str(table_input[i][myorder[j]])))
        
            self.table_flights.setEditTriggers(QtWidgets.QAbstractItemView.NoEditTriggers)        
            self.table_flights.setSizeAdjustPolicy(QtWidgets.QAbstractScrollArea.AdjustToContents)
            self.table_flights.resizeColumnsToContents()
            self.table_flights.setWordWrap(True)
            self.table_flights.setColumnWidth(3,100)

        
    def get_specific_dates(self):
        start = self.dateTimeEdit1.date().toString("yyyyMMdd")
        end = self.dateTimeEdit2.date().toString("yyyyMMdd")
        countries = []
        
        for index in self.table_operas.selectedIndexes():
            countries.append(self.opera_list[index.row()][1]) #country code
            
        countries = list(set(countries))    
        we,df = flight_search_dates(countries,start,end)
        
        table_input = we.print_flights()
        self.table_input = table_input
        self.update_flight_table()
    
        #temp = flight_search_dates(countries,start,end,departure_time="0800-2400",return_time="1000-2400")
        
        #for w in temp:
        
        
        
        
        
        
    def get_opera_weekends(self):
                
        start = self.dateTimeEdit1.date().toString("yyyyMMdd")
        end = self.dateTimeEdit2.date().toString("yyyyMMdd")
        locations = []
        
        for i,c in enumerate(self.checkboxes):
            if c.isChecked():
                locations.append(self.c_list[i])
        #print('Start: {}, End: {}\nCountries:{}'.format(start,end,locations))
        self.weekends = get_operas_in_period(start,end,locations)
        self.createTable(self.weekends)
        

    def addCheckbox(self,title='test'):
        checkbox = QtWidgets.QCheckBox(title)
        self.checkboxes.append(checkbox)
        #self.vlayout.addWidget(checkbox)
        
    def get_data_status(self):
        return 0
        #...
        #self.mystruct1 = {'A':self.lista1, 'B':self.listb1, 'C':self.listc1,'D':,'E':} 
        #return self.mystruct1

    def updateAllViews(self):
        _ = self.get_data_status()
        self.updateTable()

    def updateTable(self):
        self.tableWidget.setHorizontalHeaderLabels(['Dates','Country', 'City','Theater',"Title",'Composer'])
        #self.tableWidget.setHorizontalHeaderLabels(['A','B','C'])
        self.tableWidget.setItem(3,3,QTableWidgetItem("Updated!"))
        #self.table1.updateFromDict(self.mystruct1)
        
    def createTable(self,weekends=None):
        
        table_input=[]
        for i,weekend in enumerate(weekends):
            #print(i, weekend)
            temp = weekend.get_table()
            #print(temp)
            temp2 = []
            for j in temp:
                j.append(i)
                temp2.append(j)
            table_input.extend(temp2)
            
        self.opera_list = table_input

        rows = len(table_input)
        cols = 5 
    
        
        self.table_operas.setFont(QtGui.QFont('Arial', 8))
        #self.table_operas.setFont(QtGui.QFont('Arial', 7))
        self.table_operas.setRowCount(rows)
        self.table_operas.setColumnCount(cols)
        self.table_operas.setHorizontalHeaderLabels(['Dates','Country', 'City','Theater',"Title",'Composer'])
        #cols 6,7,8 = code, opera_index, weekend_index
        for i in range(rows): 
            for j in range(cols):
                #print(table_input[i][j])
                self.table_operas.setItem(i,j,QTableWidgetItem(str(table_input[i][j])))
        self.table_operas.setEditTriggers(QtWidgets.QAbstractItemView.NoEditTriggers)        
           
        self.table_operas.setSizeAdjustPolicy(QtWidgets.QAbstractScrollArea.AdjustToContents)
        self.table_operas.resizeColumnsToContents()

    
    

    def get_opera_info(self):
        codes = []
        op_index = []
        #weekend_index = []
        for index in self.table_operas.selectedIndexes():
            op_index.append([self.opera_list[index.row()][7],self.opera_list[index.row()][8]])
        
        self.popups = []
        
        for oi,wi in op_index:
            w = self.weekends[wi]        
            o = w[oi].getInfo()
            title = w[oi].title
            #print(o)
            self.popups.append(textView(title,o))
            self.popups[-1].show()
                
    
    
    def click_search(self):
        cities = []
        we_index = []
        #weekend_index = []
        for index in self.table_flights.selectedIndexes():
            cities.append(self.opera_list[index.row()][2])
            we_index.append(self.opera_list[index.row()][8])
            
        if citites == []:
            citites = None
        
        
        
        
    def onclick(self):
                
        row = self.table_flights.currentItem().row()
        print("row=",row)
        col = self.table_flights.currentItem().column()
        print("col=",col)
        item = self.table_flights.horizontalHeaderItem(col).text()
        print("item=",item)
        
        
    def onClick_btn(self):
        for index in self.table_flights.selectedIndexes():
            print(index.row())
            print(self.opera_list[index.row()])
            print(self.opera_list[index.row(),6])
            
            
        #self.opera_list = table_input
            
        return self.table_flights.selectedIndexes()
        
    
    def onClicked(self):
        print("Date1 {}")
        print(dateTimeEdit1.date())
        print("Date2 {}".format(dateTimeEdit2.date()))
        
        for i, checkbox in enumerate(self.checkboxes):
            if checkbox.isChecked():
                print("print {} on the screen".format(i))
                
class textView(QWidget):
    def __init__(self, title='fail', text='fail'):
        super(textView, self).__init__()
        
        self.theText = QLabel(text, self ) 
        self.setWindowTitle(title)
        self.theText.setOpenExternalLinks(True)
        self.theText.setTextFormat(QtCore.Qt.RichText)
        self.theText.setContentsMargins(2,2,2,2)
        
        self.theText.adjustSize()
        
        
        #QtWidgets.QLabel()
        

#use this for jupyter
app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
#application = QtWidgets.QApplication(sys.argv)
window = Window()
#window.setWindowTitle('Hello')
#window.resize(250, 180)
window.show()
sys.exit(app.exec_())  

SystemExit: 0

/home/larkaa/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## From designer

In [23]:
# -*- coding: utf-8 -*-
from PyQt5 import QtCore, QtGui, QtWidgets

class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(994, 638)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.frame = QtWidgets.QFrame(self.centralwidget)
        self.frame.setGeometry(QtCore.QRect(30, 40, 291, 291))
        self.frame.setFrameShape(QtWidgets.QFrame.StyledPanel)
        self.frame.setFrameShadow(QtWidgets.QFrame.Raised)
        self.frame.setObjectName("frame")
        self.label = QtWidgets.QLabel(self.frame)
        self.label.setGeometry(QtCore.QRect(0, 0, 111, 16))
        self.label.setObjectName("label")
        self.widget = QtWidgets.QWidget(self.frame)
        self.widget.setGeometry(QtCore.QRect(0, 20, 281, 249))
        self.widget.setObjectName("widget")
        self.verticalLayout_2 = QtWidgets.QVBoxLayout(self.widget)
        self.verticalLayout_2.setContentsMargins(0, 0, 0, 0)
        self.verticalLayout_2.setObjectName("verticalLayout_2")
        self.gridLayout = QtWidgets.QGridLayout()
        self.gridLayout.setObjectName("gridLayout")
        self.gridLayout_3 = QtWidgets.QGridLayout()
        self.gridLayout_3.setObjectName("gridLayout_3")
        self.btn_search_dates = QtWidgets.QPushButton(self.widget)
        self.btn_search_dates.setObjectName("btn_search_dates")
        self.gridLayout_3.addWidget(self.btn_search_dates, 0, 1, 1, 1)
        self.btn_search_weekend = QtWidgets.QPushButton(self.widget)
        self.btn_search_weekend.setObjectName("btn_search_weekend")
        self.gridLayout_3.addWidget(self.btn_search_weekend, 0, 0, 1, 1)
        self.dateEdit_2 = QtWidgets.QDateEdit(self.widget)
        self.dateEdit_2.setObjectName("dateEdit_2")
        self.gridLayout_3.addWidget(self.dateEdit_2, 1, 1, 1, 1)
        self.dateEdit = QtWidgets.QDateEdit(self.widget)
        self.dateEdit.setObjectName("dateEdit")
        self.gridLayout_3.addWidget(self.dateEdit, 1, 0, 1, 1)
        self.gridLayout.addLayout(self.gridLayout_3, 2, 0, 1, 1)
        self.verticalLayout_2.addLayout(self.gridLayout)
        spacerItem = QtWidgets.QSpacerItem(40, 9, QtWidgets.QSizePolicy.Expanding, QtWidgets.QSizePolicy.Minimum)
        self.verticalLayout_2.addItem(spacerItem)
        self.gridLayout_2 = QtWidgets.QGridLayout()
        self.gridLayout_2.setObjectName("gridLayout_2")
        self.checkBox = QtWidgets.QCheckBox(self.widget)
        self.checkBox.setObjectName("checkBox")
        self.gridLayout_2.addWidget(self.checkBox, 0, 0, 1, 1)
        self.checkBox_2 = QtWidgets.QCheckBox(self.widget)
        self.checkBox_2.setObjectName("checkBox_2")
        self.gridLayout_2.addWidget(self.checkBox_2, 1, 0, 1, 1)
        self.verticalLayout_2.addLayout(self.gridLayout_2)
        self.frame_2 = QtWidgets.QFrame(self.centralwidget)
        self.frame_2.setGeometry(QtCore.QRect(330, 40, 571, 291))
        self.frame_2.setFrameShape(QtWidgets.QFrame.StyledPanel)
        self.frame_2.setFrameShadow(QtWidgets.QFrame.Raised)
        self.frame_2.setObjectName("frame_2")
        self.widget1 = QtWidgets.QWidget(self.frame_2)
        self.widget1.setGeometry(QtCore.QRect(20, 10, 531, 271))
        self.widget1.setObjectName("widget1")
        self.verticalLayout = QtWidgets.QVBoxLayout(self.widget1)
        self.verticalLayout.setContentsMargins(0, 0, 0, 0)
        self.verticalLayout.setObjectName("verticalLayout")
        self.horizontalLayout = QtWidgets.QHBoxLayout()
        self.horizontalLayout.setObjectName("horizontalLayout")
        self.btn_search_flights = QtWidgets.QPushButton(self.widget1)
        self.btn_search_flights.setObjectName("btn_search_flights")
        self.horizontalLayout.addWidget(self.btn_search_flights)
        self.btn_get_info = QtWidgets.QPushButton(self.widget1)
        self.btn_get_info.setObjectName("btn_get_info")
        self.horizontalLayout.addWidget(self.btn_get_info)
        self.verticalLayout.addLayout(self.horizontalLayout)
        self.label_2 = QtWidgets.QLabel(self.widget1)
        self.label_2.setObjectName("label_2")
        self.verticalLayout.addWidget(self.label_2)
        self.tabWidget = QtWidgets.QTabWidget(self.widget1)
        self.tabWidget.setObjectName("tabWidget")
        self.Operas = QtWidgets.QWidget()
        self.Operas.setObjectName("Operas")
        self.table_operas = QtWidgets.QTableWidget(self.Operas)
        self.table_operas.setGeometry(QtCore.QRect(10, 10, 411, 161))
        self.table_operas.setObjectName("table_operas")
        self.table_operas.setColumnCount(0)
        self.table_operas.setRowCount(0)
        self.tabWidget.addTab(self.Operas, "")
        self.Flights = QtWidgets.QWidget()
        self.Flights.setObjectName("Flights")
        self.table_flights = QtWidgets.QTableWidget(self.Flights)
        self.table_flights.setGeometry(QtCore.QRect(10, 20, 491, 151))
        self.table_flights.setObjectName("table_flights")
        self.table_flights.setColumnCount(0)
        self.table_flights.setRowCount(0)
        self.tabWidget.addTab(self.Flights, "")
        self.verticalLayout.addWidget(self.tabWidget)
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 994, 20))
        self.menubar.setObjectName("menubar")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        self.tabWidget.setCurrentIndex(1)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.label.setText(_translate("MainWindow", "Opera Search"))
        self.btn_search_dates.setText(_translate("MainWindow", "Search Specific Dates"))
        self.btn_search_weekend.setText(_translate("MainWindow", "Search Weekends"))
        self.checkBox.setText(_translate("MainWindow", "CheckBox"))
        self.checkBox_2.setText(_translate("MainWindow", "CheckBox"))
        self.btn_search_flights.setText(_translate("MainWindow", "Search Flights"))
        self.btn_get_info.setText(_translate("MainWindow", "Get Opera Info"))
        self.label_2.setText(_translate("MainWindow", "Results"))
        self.tabWidget.setTabText(self.tabWidget.indexOf(self.Operas), _translate("MainWindow", "Operas"))
        self.tabWidget.setTabText(self.tabWidget.indexOf(self.Flights), _translate("MainWindow", "Flights"))




'''if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())
'''
    
app = QCoreApplication.instance()
if app is None:
    app = QtWidgets.QApplication(sys.argv)
#application = QtWidgets.QApplication(sys.argv)
MainWindow = QtWidgets.QMainWindow()
ui = Ui_MainWindow()
ui.setupUi(MainWindow)
MainWindow.show()
sys.exit(app.exec_())  

SystemExit: 0

In [28]:
from PyQt5 import QtGui
from PyQt5.QtWidgets import QApplication, QWidget, QCheckBox
import sys

app = QApplication(sys.argv)
w = QWidget()
w.resize(250, 150)
w.move(300, 300)
w.setWindowTitle('PyQT !!!')
w.show()

cb = QCheckBox('enlarge! window!')
cb.stateChanged.connect(lamb
                        
    

    
sys.exit(app.exec_())

SyntaxError: invalid syntax (<ipython-input-28-e4f8a8299652>, line 18)

In [25]:
from PyQt5.QtCore import QDateTime, Qt, QTimer
from PyQt5.QtWidgets import (QApplication, QCheckBox, QComboBox, QDateTimeEdit,
        QDial, QDialog, QGridLayout, QGroupBox, QHBoxLayout, QLabel, QLineEdit,
        QProgressBar, QPushButton, QRadioButton, QScrollBar, QSizePolicy,
        QSlider, QSpinBox, QStyleFactory, QTableWidget, QTabWidget, QTextEdit,
        QVBoxLayout, QWidget)

import sys

class WidgetGallery(QDialog):
    def __init__(self, parent=None):
        super(WidgetGallery, self).__init__(parent)

        self.originalPalette = QApplication.palette()

        styleComboBox = QComboBox()
        styleComboBox.addItems(QStyleFactory.keys())

        styleLabel = QLabel("&Style:")
        styleLabel.setBuddy(styleComboBox)

        self.useStylePaletteCheckBox = QCheckBox("&Use style's standard palette")
        self.useStylePaletteCheckBox.setChecked(True)

        disableWidgetsCheckBox = QCheckBox("&Disable widgets")

        self.createTopLeftGroupBox()
        self.createTopRightGroupBox()
        self.createBottomLeftTabWidget()
        self.createBottomRightGroupBox()
        self.createProgressBar()

        styleComboBox.activated[str].connect(self.changeStyle)
        self.useStylePaletteCheckBox.toggled.connect(self.changePalette)
        disableWidgetsCheckBox.toggled.connect(self.topLeftGroupBox.setDisabled)
        disableWidgetsCheckBox.toggled.connect(self.topRightGroupBox.setDisabled)
        disableWidgetsCheckBox.toggled.connect(self.bottomLeftTabWidget.setDisabled)
        disableWidgetsCheckBox.toggled.connect(self.bottomRightGroupBox.setDisabled)

        topLayout = QHBoxLayout()
        topLayout.addWidget(styleLabel)
        topLayout.addWidget(styleComboBox)
        topLayout.addStretch(1)
        topLayout.addWidget(self.useStylePaletteCheckBox)
        topLayout.addWidget(disableWidgetsCheckBox)

        mainLayout = QGridLayout()
        mainLayout.addLayout(topLayout, 0, 0, 1, 2)
        mainLayout.addWidget(self.topLeftGroupBox, 1, 0)
        mainLayout.addWidget(self.topRightGroupBox, 1, 1)
        mainLayout.addWidget(self.bottomLeftTabWidget, 2, 0)
        mainLayout.addWidget(self.bottomRightGroupBox, 2, 1)
        mainLayout.addWidget(self.progressBar, 3, 0, 1, 2)
        mainLayout.setRowStretch(1, 1)
        mainLayout.setRowStretch(2, 1)
        mainLayout.setColumnStretch(0, 1)
        mainLayout.setColumnStretch(1, 1)
        self.setLayout(mainLayout)

        self.setWindowTitle("Styles")
        self.changeStyle('Windows')

    def changeStyle(self, styleName):
        QApplication.setStyle(QStyleFactory.create(styleName))
        self.changePalette()

    def changePalette(self):
        if (self.useStylePaletteCheckBox.isChecked()):
            QApplication.setPalette(QApplication.style().standardPalette())
        else:
            QApplication.setPalette(self.originalPalette)

    def advanceProgressBar(self):
        curVal = self.progressBar.value()
        maxVal = self.progressBar.maximum()
        self.progressBar.setValue(curVal + (maxVal - curVal) / 100)

    def createTopLeftGroupBox(self):
        self.topLeftGroupBox = QGroupBox("Group 1")
        
        self.checkboxes = []
        self.c_list = ['Czech Republic','Denmark','Estonia','Finland',
         'France','Germany','Greece','Hungary','Iceland',
         'Ireland','Italy','Latvia','Lithuania','Netherlands',
         'Norway','Poland','Portugal','Romania','Slovakia',
         'Slovenia','Spain','Sweden','Ukraine','United Kingdom' ]
        
        
        

        radioButton1 = QRadioButton("Radio button 1")
        radioButton2 = QRadioButton("Radio button 2")
        radioButton3 = QRadioButton("Radio button 3")
        radioButton1.setChecked(True)

        checkBox = QCheckBox("Tri-state check box")
        checkBox.setTristate(True)
        checkBox.setCheckState(Qt.PartiallyChecked)

        layout = QVBoxLayout()
        layout.addWidget(radioButton1)
        layout.addWidget(radioButton2)
        layout.addWidget(radioButton3)
        layout.addWidget(checkBox)
        layout.addStretch(1)
        self.topLeftGroupBox.setLayout(layout)    

    def createTopRightGroupBox(self):
        self.topRightGroupBox = QGroupBox("Group 2")

        defaultPushButton = QPushButton("Default Push Button")
        defaultPushButton.setDefault(True)

        togglePushButton = QPushButton("Toggle Push Button")
        togglePushButton.setCheckable(True)
        togglePushButton.setChecked(True)

        flatPushButton = QPushButton("Flat Push Button")
        flatPushButton.setFlat(True)

        layout = QVBoxLayout()
        layout.addWidget(defaultPushButton)
        layout.addWidget(togglePushButton)
        layout.addWidget(flatPushButton)
        layout.addStretch(1)
        self.topRightGroupBox.setLayout(layout)

    def createBottomLeftTabWidget(self):
        self.bottomLeftTabWidget = QTabWidget()
        self.bottomLeftTabWidget.setSizePolicy(QSizePolicy.Preferred,
                QSizePolicy.Ignored)

        tab1 = QWidget()
        tableWidget = QTableWidget(10, 10)

        tab1hbox = QHBoxLayout()
        tab1hbox.setContentsMargins(5, 5, 5, 5)
        tab1hbox.addWidget(tableWidget)
        tab1.setLayout(tab1hbox)

        tab2 = QWidget()
        textEdit = QTextEdit()

        textEdit.setPlainText("Twinkle, twinkle, little star,\n"
                              "How I wonder what you are.\n" 
                              "Up above the world so high,\n"
                              "Like a diamond in the sky.\n"
                              "Twinkle, twinkle, little star,\n" 
                              "How I wonder what you are!\n")

        tab2hbox = QHBoxLayout()
        tab2hbox.setContentsMargins(5, 5, 5, 5)
        tab2hbox.addWidget(textEdit)
        tab2.setLayout(tab2hbox)

        self.bottomLeftTabWidget.addTab(tab1, "&Table")
        self.bottomLeftTabWidget.addTab(tab2, "Text &Edit")

    def createBottomRightGroupBox(self):
        self.bottomRightGroupBox = QGroupBox("Group 3")
        self.bottomRightGroupBox.setCheckable(True)
        self.bottomRightGroupBox.setChecked(True)

        lineEdit = QLineEdit('s3cRe7')
        lineEdit.setEchoMode(QLineEdit.Password)

        spinBox = QSpinBox(self.bottomRightGroupBox)
        spinBox.setValue(50)

        dateTimeEdit = QDateTimeEdit(self.bottomRightGroupBox)
        dateTimeEdit.setDateTime(QDateTime.currentDateTime())

        slider = QSlider(Qt.Horizontal, self.bottomRightGroupBox)
        slider.setValue(40)

        scrollBar = QScrollBar(Qt.Horizontal, self.bottomRightGroupBox)
        scrollBar.setValue(60)

        dial = QDial(self.bottomRightGroupBox)
        dial.setValue(30)
        dial.setNotchesVisible(True)

        layout = QGridLayout()
        layout.addWidget(lineEdit, 0, 0, 1, 2)
        layout.addWidget(spinBox, 1, 0, 1, 2)
        layout.addWidget(dateTimeEdit, 2, 0, 1, 2)
        layout.addWidget(slider, 3, 0)
        layout.addWidget(scrollBar, 4, 0)
        layout.addWidget(dial, 3, 1, 2, 1)
        layout.setRowStretch(5, 1)
        self.bottomRightGroupBox.setLayout(layout)

    def createProgressBar(self):
        self.progressBar = QProgressBar()
        self.progressBar.setRange(0, 10000)
        self.progressBar.setValue(0)

        timer = QTimer(self)
        timer.timeout.connect(self.advanceProgressBar)
        timer.start(1000)



        
app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
ex = WidgetGallery()
ex.show()
sys.exit(app.exec_())



SystemExit: 0

In [1]:
import tkinter as tk
import sys

class Window(tk.Frame):
    
    def __init__(self,master=None):
        tk.Frame.__init__(self,master)
        self.master = master
        
        self.init_window()
        
    def init_window(self):
        self.master.title("GUI")
        self.pack(fill=tk.BOTH, expand=1) 
        
        menu = tk.Menu(self.master)
        self.master.config(menu=menu)
        
        file = tk.Menu(menu)
        file.add_command(label='Exit',command = self.master.destroy)
        menu.add_cascade(label='File', menu=file)
        
        edit = tk.Menu(menu)
        edit.add_command(label='Undo')
        menu.add_cascade(label='Edit',menu=edit)
        
        #quitButton = tk.Button(self, text = 'Quit', command = self.client_exit)
        #quitButton.place(x=0,y=0)
        
    def client_exit(self):
        tk.quit()
        
        
root = tk.Tk()
root.geometry("400x300")
app = Window(root)
root.mainloop()

In [42]:

test = 'https://www.operabase.com/ressources/views/festivals/season_prod_fetch.php?id=93759&statut=&lang=en'

# choosing year periods
test = 'https://www.operabase.com/ressources/views/companies/season_period_fetch.php?id=787&lang=en'


test = 'https://www.operabase.com/ressources/views/festivals/season_prod_fetch.php?id=120008&statut=&lang=en'

#get a calendar view of an opera house
test = 'https://www.operabase.com/ressources/views/companies/season_fetch_calendar.php?season=0801&code=snosn&ensid=no.Den+Norske+Opera&year=2019&statut=&lang=en'

# get list of performances from one opera house
test = 'https://www.operabase.com/ressources/views/companies/season_fetch.php?season=0801&code=snosn&ensid=no.Den+Norske+Opera&year=2019&statut=&lang=en'



payload = 'country_iso=wd&city=&company=&company_name=&dotL=&comp_id=&comp_name=&work_id=&work_name=&artist_name=&artist_id=&from=20190802&to=20191113&default_venue='

#"/ressources/views/companies/season_fetch_calendar.php",{season :season, code : code, ensid : ensid, year : year,statut : statut, lang : lang}, function( data )
#120008

s = requests.session()

#iso 
#"w" - western europe
#"e" - eastern europe
#"wd" - world
#"s" - scandinavia
url = 'https://www.operabase.com/en'
data = {"country_iso": "wd","from":"20190927","to":"20190930"}
r = s.post(url, data = data)

soup = BeautifulSoup(r.text, "lxml")

In [ ]:
# GET PASSWORDS


url='https://www.operabase.com/ressources/views/companies/companies_fetch_code.php'
s = get_tor_session()
res = []
for i in comp_codes.values.tolist():
    payload = {'id':str(i[0])}
    r = s.post(url,data=payload)
    res.append(r.text)
    
# MERGE PASSWORDS
def cleanhtml(raw_html):
    #cleanr = re.compile('<.*?>')
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return cleantext

codes = res
temp = []
comp_ids = comp_codes.values.tolist()

for i in range(len(codes)):
    z = cleanhtml(codes[i]).strip().split(' ')
    if len(z)==4:
        a = z[1]
        b = z[3]
        temp.append([comp_ids[i][0], comp_ids[i][1], a, b])
    else:
        temp.append([comp_ids[i][0], comp_ids[i][1], '', ''])

    
comp_names_pass = pd.DataFrame(temp,columns=['id','theater','user','pwd'])
#print(comp_names_pass.tail())

comp_names_pass.to_csv('company_codes_and_passwords.csv',index=False)

## GOOGLE FLIGHTS

In [22]:
time,company,duration,codes,direct,price,_ = aller.split('\n')

['22:25 – 23:00',
 'Ryanair',
 '1 h 35 min',
 'BVA–DUB',
 'Sans escale',
 '561 €',
 'aller-retour']

## eurostar

In [65]:
origin = '8727100'
dest = '7015400'
adults = '1'
outbound_date = '2019-09-13'
inbound_date = '2019-09-15'

url ='https://booking.eurostar.com/fr-fr/train-search?origin={}&destination={}&adult={}&outbound-date={}&inbound-date={}'.format(
    origin, dest,adults,outbound_date,inbound_date)

url

'https://booking.eurostar.com/fr-fr/train-search?origin=8727100&destination=7015400&adult=1&outbound-date=2019-09-13&inbound-date=2019-09-15'

## functions found

In [ ]:
*()
 * Author : Armand Arapian
 */

 $(document).ready(function(){ 
    fancybox_iframe('iframe')
    fancybox_iframe('fancybox')
    fancybox_iframe_big('fancybox_big')
    (/how, many, days, since, 0108, -4, years, -, 4, ans)
    var today=new Date();
    var currentYear=today.getFullYear();
    var currentMonth=today.getMonth();
    var august = new Date(currentYear-3, 7, 1) // (1, 1 stands of 1st of January)
    var dateDifference = today - august;
    dateDifference=dateDifference / (1000 * 60 *60 *24);
    dateDifference=Math.round(dateDifference);
    dateDifference=dateDifference-1
    same_width();
    
    $('#reset').click(function () {
        $('input').val('');
    })
    $('#calendar').click(function () {
        if($('#no_calendar').css('display')=='block'){
            $('#no_calendar,.ct_list').fadeOut();
            $('#cal_view,#list,#print').fadeIn();
        }        
        $('#cal').trigger('sortReset');
        return false;
    }) 
    
    $('#list').click(function () {
        if($('#no_calendar').css('display')=='none'){
            $('#no_calendar,.ct_list').fadeIn();
            $('#cal_view,#print,#list').fadeOut();
        }
        
        $('#cal').trigger('sortReset');
        return false;
    }) 
    $("#cal thead th:eq(0)").data("sorter", false);
    $("#cal").tablesorter({
        
    });
    $('#seven_more').click(function(){
        var to = $('#to').val();
        var from = $('#from').val();
        var year = to.substring(0,4)
        var month = to.substr(4,2)
        var day = to.substr(6,2)
        
        var date = year+'-'+month+'-'+day;
        (/alert(day+'/'+month+'/'+year+', '+addDays(date,, '7')))
        $('#to').val(addDays(date, '7'));
        $('#from').val(from);
        $('#form1').submit();
    });
    $('#back2comp').click(function(){
        $('#back2comp,#cal_view').hide();
        $('#125_result,#250_result').hide();
        $('.companies_work_bordered,#cities_prod,.list_title_medium,#no_calendar').fadeIn();
    });
    $('#print').click(function(){        
        $("#cal_view").print({
            (/Use, Global, styles)
            globalStyles : true,
            (/Add, link, with, attrbute, media=print)
            mediaPrint : false,
            (/Custom, stylesheet)
            stylesheet : 'https://operabase.com/ressources/css/home.css',
            (/Print, in, a, hidden, iframe)
            iframe : true,
            (/Don't, print, this)
            noPrintSelector : ".avoid-this",
            (/Add, this, at, top)
            prepend : false,
            (/Add, this, on, bottom)
            append : false,
            (/Log, to, console, when, printing, is, done, via, a, deffered, callback)
            (/deferred:, $.Deferred().done(function(), {, console.log('Printing, done',, arguments);, }))
        });
    })
    $('.bordered_line_noborder').prepend('<img id="perfs_back"  class="inline" src="/ressources/pix/icons/back2search.png" title="Back to search"/><br />');
    $('#perfs_back').click(function () {
        $('#form1_back').submit();    
    })
    $('.fest_info_company').click(function () {
        $('#company_info').fadeToggle();
        $('#booking_info').hide();
    })
    (/calendars)
    $.post('/ressources/php/fetch_auth.php',{},function(data){
        var auth = $.trim(data);   
        var dateToday = new Date(); 
        if (auth == 'W' || auth == 'A' || auth == 'S'|| auth == 'D' || auth == 'J'|| auth == 'O'|| auth == 'M' ) { 
            var dateFormat = "dd/mm/yy",  
            
            from = $( ".from_date" ).datepicker({
                    defaultDate: "+1w",
                    changeMonth: true,
                    dateFormat: 'dd/mm/yy',      
                    changeYear: true,
                    numberOfMonths: 1,
                    showOtherMonths: true,
                    selectOtherMonths: true,
                    yearRange: "1996:"+ (dateToday.getFullYear() + 5)
                })
                .on( "change", function() {
                    to.datepicker( "option", "minDate", getDate( this ) );
                }),
            to = $( ".to_date" ).datepicker({
                defaultDate: "+1w",
                changeMonth: true, 
                dateFormat: 'dd/mm/yy',     
                changeYear: true,
                numberOfMonths: 1,
                showOtherMonths: true,
                selectOtherMonths: true,
                yearRange: "1996:"+ (dateToday.getFullYear() + 5)
            })
            
        } else {
            var dateFormat = "dd/mm/yy",        
            from = $( ".from_date" ).datepicker({
                defaultDate: "+1w",
                changeMonth: true,
                minDate: -dateDifference,
                dateFormat: 'dd/mm/yy',      
                changeYear: true,
                numberOfMonths: 1,
                showOtherMonths: true,
                selectOtherMonths: true
            })
            .on( "change", function() {
                to.datepicker( "option", "minDate", getDate( this ) );
            }),
            to = $( ".to_date" ).datepicker({
                defaultDate: "+1w",
                changeMonth: true, 
                minDate: -dateDifference,
                dateFormat: 'dd/mm/yy',     
                changeYear: true,
                numberOfMonths: 1,
                showOtherMonths: true,
                selectOtherMonths: true
            })
            .on( "change", function() {
                from.datepicker( "option", "maxDate", getDate( this ) );
            });
        }

        function getDate( element ) {
            var date;
            try {
                date = $.datepicker.parseDate( dateFormat, element.value );
            } catch( error ) {
                date = null;
            }
            return date;
        }
        if($( "#country_city" ).length>0) {
            $( "#country_city" ).autocomplete({
                (/minLength:, 2,)
                source: '/ressources/views/country/select_country_city.php?lang='+lang,
                autoFocus: true,
                select: function( event, ui ) {
                    $( "#country_iso").val(ui.item.iso)
                    var country_ok= ui.item.country_ok
                    var iso = $( "#country_iso").val();
                    if(country_ok==='1'){    
                        $( "#country_home,#country_city" ).val(ui.item.country)  
                        $( "#city_home,#city" ).val('')         
                    }else{    
                        $( "#country_home,#country_city" ).val(ui.item.country_list)                         
                        $( "#city_home,#city" ).val(ui.item.country)     
                    }
                    if($( ".accueil_dates_go" ).css('display')=='none')
                        $( ".accueil_dates_go" ).fadeIn();
                    return false;
                }
            }).data( "ui-autocomplete" )._renderItem = function( ul, item ) {
                var style = '';var iso = '';
                if(item.country_ok==='1')
                    style = 'font-weight: bold'
                if(item.iso=='')
                    iso = '';
                else
                    iso = "("+item.iso+")";
                return $( "<li>" )
                .append( "<a style=\""+style+"\">" + item.country_list +iso+" </a>" )
                .appendTo( ul )
                .css('font-size','10px');
            }
        }
	if($( "#city_home" ).length>0) { 					
            $( "#city_home" ).autocomplete({
                (/minLength:, 2,)
                search:function(event,ui){
                    var newUrl='/ressources/views/cities/select_city.php?c='+$( "#country_iso").val()+'&lang='+lang;
                    $(this).autocomplete("option","source",newUrl)
                },
                autoFocus: true,
                source:[],
                select: function( event, ui ) {
                    var city = ui.item.original;
                    var country = ui.item.country;
                    var iso = ui.item.iso;
                    $( "#country_home" ).val(ui.item.country)
                    $( "#country_iso").val(ui.item.iso)
                    $( "#city_home,#city" ).val(city)
                    if($( ".accueil_dates_go" ).css('display')=='none')
                        $( ".accueil_dates_go" ).fadeIn();
                    return false;
                }
            }).data( "ui-autocomplete" )._renderItem = function( ul, item ) {
                return $( "<li>" )
                        .append( "<a>" + item.city +" ("+item.country+")</a>" )
                        .appendTo( ul )
                        .css('font-size','12px');
            }
        }
        if($( "#company_home" ).length>0) {   
            $( "#company_home" ).autocomplete({
                (/minLength:, 2,)
                search:function(event,ui){
                    var newUrl='/ressources/views/companies/select_ensemble2.php?c='+$( "#country_iso").val()+'&ct='+$( "#city" ).val()+'&lang='+lang;
                    $(this).autocomplete("option","source",newUrl)
                },
                autoFocus: true,
                source: [],                
                select: function( event, ui ) {
                    var ensemble = ui.item.ensemble;
                    var id = ui.item.id;
                    $( "#company_home,#company_name" ).val( ensemble );
                    $( "#company" ).val( ui.item.dotE );
                    (/$(, "#country_iso", ).val(, ui.item.iso, );)
                    if($( ".accueil_dates_go" ).css('display')=='none')
                        $( ".accueil_dates_go" ).fadeIn();
                    return false;
                }
            }).data( "ui-autocomplete" )._renderItem = function( ul, item ) {
                return $( "<li>" )
                        .append( "<a>" + item.ensemble +" ("+item.country+")</a>" )
                        .appendTo( ul )
                        .css('font-size','12px');
            }
        }
        if($( "#venue_home" ).length>0) { 
            $( "#venue_home" ).autocomplete({
                (/minLength:, 2,)
                search:function(event,ui){
                    var newUrl='/ressources/views/venues/select_venues2.php?c='+$( "#country_iso").val()+'&ct='+$( "#city" ).val()+'&company='+$( "#company").val()+'&lang='+lang;
                    $(this).autocomplete("option","source",newUrl)
                },
                autoFocus: true,
                source: [],
                select: function( event, ui ) {
                    var venue = ui.item.venue;
                    var dotL = ui.item.dotL;
                    var iso = ui.item.iso;
                    var country = ui.item.country;
                    var city = ui.item.city;
                    $( "#country_home" ).val( country );
                    $( "#country_iso" ).val( iso );
                    $( "#venue_home" ).val( venue );
                    $( "#city_home" ).val( city );
                    $( "#city" ).val( city );
                    $( "#dotL" ).val( dotL );
                    $( "#choice_row3" ).fadeTo( "slow", 0.33 );
                    $( "#company_home" ).prop('disabled','disabled')
                    if($( ".accueil_dates_go" ).css('display')=='none')
                        $( ".accueil_dates_go" ).fadeIn();
                    return false;
                }
            }).data( "ui-autocomplete" )._renderItem = function( ul, item ) {
                if(item.salle !='') 
                    var salle = ', '+item.salle
                else
                    var salle = '';
                return $( "<li>" )
                        .append( "<a>" + item.venue +salle+" ("+item.city+", "+item.country+") </a>" )
                        .appendTo( ul )
                        .css('font-size','12px');
            }
        }
        if($( "#composer_home" ).length>0) {     
            $( "#composer_home" ).autocomplete({
                (/minLength:, 2,)
                search:function(event,ui){
                    var newUrl='/ressources/views/composers/select_composer.php?c=&ct=&company=&dotL=&lang='+lang;
                    $(this).autocomplete("option","source",newUrl)
                },
                autoFocus: true,
                source: [],
                select: function( event, ui ) {
                    var final_display = ui.item.final_display;
                    var id_comp = ui.item.id_comp;
                    $( "#composer_home,#comp_name" ).val( final_display );
                    $( "#comp_id" ).val( id_comp );
                    if($( ".accueil_dates_go" ).css('display')=='none')
                        $( ".accueil_dates_go" ).fadeIn();
                    return false;
                }
            }).data( "ui-autocomplete" )._renderItem = function( ul, item ) {
                return $( "<li>" )
                        .append( "<a>" + item.first_name +" "+item.name+" </a>" )
                        .appendTo( ul )
                        .css('font-size','12px');
            }
        }
        if($( "#title_home" ).length>0) {   
            $( "#title_home" ).autocomplete({
                (/minLength:, 2,)
                search:function(event,ui){
                    var newUrl='/ressources/views/works/select_work2.php?c=&ct=&company=&dotL=&comp_id='; 
                    $(this).autocomplete("option","source",newUrl)
                },  
                autoFocus: true,
                source: [],
                select: function( event, ui ) { 
                    var original = ui.item.original;
                    var composer = ui.item.composer;
                    var comp_id = ui.item.comp_id;
                    var work_id = ui.item.work_id;
                    $( "#title_home,#work_name" ).val( original );
                    $( "#comp_id" ).val( comp_id );
                    $( "#composer_home,#comp_name" ).val( composer );
                    $( "#work_id" ).val( work_id );
                    if($( ".accueil_dates_go" ).css('display')=='none')
                        $( ".accueil_dates_go" ).fadeIn();
                    return false;
                }
            }).data( "ui-autocomplete" )._renderItem = function( ul, item ) {
                return $( "<li>" )
                        .append( "<a>" + item.original +" ("+item.composer+") </a>" )
                        .appendTo( ul )
                        .css('font-size','12px');
            }
        }
        if($( "#artist_home" ).length>0) {  
            $( "#artist_home" ).autocomplete({
                (/minLength:, 2,)
                search:function(event,ui){
                    var newUrl='/ressources/views/artists/select_artist2.php?c='+$( "#country_iso").val()+'&ct='+$( "#city" ).val()+'&company='+$( "#company").val()+'&dotL='+$( "#dotL").val()+'&comp='+$( "#comp_id").val()+'&work='+$( "#work_id").val()+'&lang='+lang;
                    $(this).autocomplete("option","source",newUrl)
                },
                autoFocus: true,
                source:[],
                select: function( event, ui ) {
                    var full_name = ui.item.full_name;
                    var idart = ui.item.id;
                    $("#artist_home,#artist_name").val( full_name ); 
                    $("#artist_id").val( idart );
                    if($( ".accueil_dates_go" ).css('display')=='none')
                        $( ".accueil_dates_go" ).fadeIn()
                    return false;
                }
            }).data( "ui-autocomplete" )._renderItem = function( ul, item ) { 
                    
                var job = item.job;
                if(job.length>2){
                    job = ' ('+job+')';
                }

                return $( "<li>" )

                .append("<a>" + item.full_name + job +"</a>")
                .appendTo( ul )
                .css('font-size','12px');
            }
        }
        $( "#perfs_go_all" ).click(function(){
            var from = $( ".from_date" ).val();
            if(from!=''){
                var tab = from.split('/')
                from = tab[2]+tab[1]+tab[0];
            }
            var to = $( ".to_date" ).val();
            if(to!=''){
                var tab = to.split('/')
                to = tab[2]+tab[1]+tab[0];
            }
            if(from=='')
                to='';
            $('#from').val(from)
            $('#to').val(to)
            var iso = $('#country_iso').val()
            var city = $('#city').val()
            var company = $('#company').val()
            var dotL = $('#dotL').val()
            var comp_id = $('#comp_id').val()
            var work_id = $('#work_id').val()
            var artist_name = $('#artist_home').val()
            var artist_name2 = $('#artist_name').val()
            var artist_id = $('#artist_id').val()
            if(comp_id=='' && iso =='' && city=='' && company=='' && dotL=='' && dotL==''){
                $( "#country_iso" ).val('wd') ;
            
            
            }
            (/alert(iso+city))
            (/alert('from:'+from+'\nto:'+to+'\ncountry:'+iso+'\ncity:'+city+'\ncompany:'+company+'\ndotL:'+dotL+'\ncomp, id:'+comp_id+'\nwork, id:'+work_id+'\nartist, name:'+artist_name+'\nartist, name, 2:'+artist_name2+'\nartist, id:'+artist_id))
            *if((comp_id=='', &&, iso, =='', &&, city=='', &&, company=='', &&, dotL=='', &&, dotL=='', &&, artist_id=='', &&, to==''){)
                $('#country_city,#composer_home,#company_home,#title_home').prop('placeholder','?');
                
                return false;
            } */               
            $('#form1').submit();
            
        })
        if($('.list_title_medium').length>0){
            var ct = '';
            $('.list_title_medium').each(function(){
                var offset = $(this).offset();
                ct +='<div class="ct_list" data-pos="'+offset.top+'" onclick="pleaseScroll('+offset.top+')">'+$(this).text()+'<span class="verticalbar">|</span></div>';
            })
            $('#view').prepend('<div id="ch_country">'+ct+'<div class="clear_both"></div></div>')
        }
        $('.from_date').focus(function(){
            $(this).removeAttr('placeholder');
                    
        })        
        var wperiod = 0;
        $('.companies_work_period').each(function(){                    
            var w = $(this).width();
            if(wperiod<w)
                wperiod=w;                    
        });
        $('.companies_work_period').width(wperiod);
        $(".fest_info").click(function(){
            $(".festival_info").hide();
            $(this).parent().next().fadeToggle('slow')
            $('.booking_info').hide();
        }) 
        
        $('.country_city').click(function(){
            var from = $('#from').val()
            var to = $('#to').val()
            var iso = $('#country_iso').val()
            var city = $(this).data('city')
            var nb = $(this).data('nb')
            $.post('/ressources/views/perfs/cities_prog.php',{city : city, country : iso, from : from, to : to, lang : lang, auth : auth},function(data){
                (/alert(data))
                $('#prod'+nb).html(data);
                $('#prod'+nb).slideToggle()

                $(".fest_info").click(function(){
                    $(".festival_info").hide();
                    $(this).parent().next().fadeToggle('slow')
                })
            })
        })      
    })
    
        var getsearch = '';
        $( "#soundex,#exact" ).prop('checked',false)
        $( "#soundex" ).click(function(){
            $( "#exact" ).attr('checked',false)
            if($( "#soundex" ).is(':checked'))
                getsearch='soundex'
            else
                getsearch=''
            $( "#type_artist_search" ).val(getsearch);
        });
        $( "#exact" ).click(function(){
            $( "#soundex" ).attr('checked',false)
            if($( "#exact" ).is(':checked'))
                getsearch='exact'
            else
                getsearch=''
            $( "#type_artist_search" ).val(getsearch);
        });
        if($( "#artist_search" ).length>0){
        $( "#artist_search" ).autocomplete({
            search:function(event,ui){
                var newUrl='/ressources/views/artists/select_artist.php?type='+$( "#type_artist_search" ).val();
                $(this).autocomplete("option","source",newUrl)
            },
            autoFocus: true,
            source:[],
            minLength: 2,
            autoFocus: true,
            select: function( event, ui ) {
                $("#artist_search").val( ui.item.full_name );                    
                if(ui.item.id!='0'){
                    window.location.href='/a/'+ ui.item.full_name +'/'+  ui.item.id +'/'+ lang;
                }else{
                    window.location.href='/a/'+ ui.item.full_name +'/'+ lang;
                }
                return false;
            }
        }).data( "ui-autocomplete" )._renderItem = function( ul, item ) {
            var job = item.job;
            if(job.length>1){
                job = ' ('+job+')';
            }

            return $( "<li>" )
            .append("<a>" + item.full_name + job +"</a>")
            .appendTo( ul )
            .css('font-size','12px');
        }
    
    }
    if($( "#company_home2" ).length>0){
        $( "#company_home2" ).autocomplete({
            minLength: 2,
            source: '/ressources/views/companies/select_ensemble.php?lang='+lang,
            autoFocus: true,
            select: function( event, ui ) {                    
                $(this).val( ui.item.ensemble );
                window.location.href= '//'+host+'/c/'+ui.item.id +'/'+lang;
                $( "#companie_go" ).click(function(){
                    window.location.href= '//'+host+'/c/'+ui.item.id +'/'+lang;
                })
                return false;
            }
        }).data( "ui-autocomplete" )._renderItem = function( ul, item ) {
                return $( "<li>" )
                .append( "<a>" + item.ensemble +" ("+item.country+")</a>" )
                .appendTo( ul )
            } 
    }
    $('#company_home').keydown(function(e){
        if(e.keyCode == 46 || e.keyCode == 8){
            $('#company,#company_back,#company_name,#company_name_back,#default_venue,#country_iso,#country_iso_back,#city,city_back').val('');            
        }
    });
    
    $('#title_home').keydown(function(e){
        if(e.keyCode == 46 || e.keyCode == 8){
            $('#work_id,#work_id_back,#work_name,#work_name_back').val('');            
        }
    });
    

});
function pleaseScroll(pos){
    
    $("html, body").animate({
        'scrollTop': pos
    }, 500);
}

function show_prod(id,statut,lang){
    $.get( "/ressources/views/festivals/season_prod_fetch.php",{id : id,statut : statut,lang : lang}, function( data ) {
        $('#content_'+id).html(data)
        $('#content_'+id).slideToggle(5)
    })
}
function show_booking(id,venue,lang){
    $('#booking_'+id).html('<img src="/ressources/pix/icons/del.gif" class="booking_info_close" onclick="festival_info_close()"><iframe src="//www.booking.com/flexiproduct.html?product=nsb&w=100%25&h=auto&lang='+lang+'&aid=1733065&target_aid=1733065&df_num_properties=3&fid=1552502018535&ss='+venue+'" scrolling="no" style="padding: 0px; margin: 0px; overflow: hidden; width: 100%; height: 256px;" marginheight="0" marginwidth="0" allowtransparency="true" id="booking_widget__1733065__1552503769103" data-responsive="true" width="100%" height="256" frameborder="0"></iframe>')
    $('.booking_info,.festival_info').hide();
    $('#booking_'+id).slideToggle(1000)
    $('.booking_info_close').slideToggle(1000)
}
function show_booking_250(id,venue,lang){
    $('#bookings_'+id).html('<img src="/ressources/pix/icons/del.gif" class="booking_info_close" onclick="festival_info_close()"><iframe src="//www.booking.com/flexiproduct.html?product=nsb&w=100%25&h=auto&lang='+lang+'&aid=1733065&target_aid=1733065&df_num_properties=3&fid=1552502018535&ss='+venue+'" scrolling="no" style="padding: 0px; margin: 0px; overflow: hidden; width: 100%; height: 256px;" marginheight="0" marginwidth="0" allowtransparency="true" id="booking_widget__1733065__1552503769103" data-responsive="true" width="100%" height="256" frameborder="0"></iframe>')
    $('.booking_info,.festival_info').hide();
    $('#bookings_'+id).slideToggle(1000)
    $('.booking_info_close').slideToggle(1000)
}
function show_prod_tour(id,id_div,statut,lang,begin,end,dotL){
    $.get( "/ressources/views/festivals/season_prod_fetch.php",{id : id,statut : statut,lang : lang, begin : begin, end : end, dotL : dotL}, function( data ) {
        $('#content_'+id_div).html(data)
        $('#content_'+id_div).slideToggle(5)
    })
}
function festival_info_close(){
    $('.festival_info,.booking_info,.booking_info_close').fadeOut();
}
function same_width(){ 
    var wperiod = 0;
    $('.companies_work_period').each(function(){                    
        var w = $(this).width();
        if(wperiod<w)
            wperiod=w;                    
    });
    $('.companies_work_period').width(wperiod);
}

function show_day(){
    $('.jour_date').fadeToggle();
}
function fancybox_iframe(e){
    $("."+e).fancybox({
        type            : "iframe",
        maxWidth        : 1900,
        maxHeight       : 900,
        fitToView       : true,
        width           : '80%',
        height          : '110%',
        autoSize	: false,
        closeClick	: false,
        openEffect	: 'none',
        closeEffect	: 'none'
    });
}
function fetch_seasons(season,code,ensid,year,statut,lang,np,wp){
    $('.companies_refresh').css('cursor','wait')
    $('body').addClass('wait')
    $.getJSON( "companies/season_fetch.php",{season :season, code : code, ensid : ensid, year : year,statut : statut, lang : lang}, function( data ) {
        var items = [];
        $.each( data, function(i, item) {
            var class_type = 'companies_around_WP_empty';
            var prodtype = '';
            if(item.prodtype.match(/P/)){
                
                prodtype = '<img src="pix/icons/wp.png" class="companies_icon_wpprod" title="'+wp+'" alt="'+wp+'" />';
            }else if(item.prodtype.match(/!/)){
                 prodtype = '<img src="pix/icons/newprod.png" class="companies_icon_newprod" title="'+np+'" alt="'+np+'" />';
            }
            (/$('.companies_refresh').remove();)
            $('body').removeClass('wait')
            (/$('body').prepend('<img, src="pix/icons/refresh.jpg", class="companies_refresh", title="Refresh, the, page", onclick="fetch_seasons(\''+season+'\',\''+code+'\',\''+ensid+'\',\''+year+'\',\''+statut+'\',\''+lang+'\')"/>'))
            $('.companies_refresh').css('cursor','default')
            
            if(statut == 'W' || statut == 'A')
                var apply_admin = '<a href="season_apply.php?id='+item.id_prod+'" class="iframe"><img src="http://operabase.com/pix/icons/chgsu.gif" class="companies_apply companies_icons inline" data-id_prod="'+item.id_prod+'" title="Edit '+item.title+'"/></a>';
            else
                var apply_admin ='';
            
            if(statut == 'W' || statut == 'A'){
                var theatre_updt = '<a href="season_updt.php?id='+item.id_prod+'&edit=&lang=<?php echo $lang;?>&user=<?php echo $user;?>" class="iframe" data-fancybox-type="iframe"><img src="http://operabase.com/pix/icons/chg.gif" class="companies_updt_theatre companies_icons inline" data-id_prod="'+item.id_prod+'&lang=<?php echo $lang;?>&user=<?php echo $user;?>" title="Edit '+item.title+'"/></a>';
                var euphonia = '<img src="http://operabase.com/pix/icons/bookaddk.gif" class="companies_euphonia  inline" data-id_prod="'+item.id_prod+'"/>';
                var del = '<img src="http://operabase.com/pix/icons/del.gif" class="del companies_icons inline" data-id_prod="'+item.id_prod+'" title="Delete '+item.title+'" style="float: right" onclick="del(\''+item.id_prod+'\')"/>';
            }else{
                var theatre_updt ='';
                var euphonia = '';
                var del = '';
            }
            var is_cast = '';
            var is_cond = '';
            var is_prod = '';
            var is_location = '';
            var is_info = '';
            var is_nn = '';
            var is_role = '';
            var pluriel = '';
            if(statut == 'W' || statut == 'A'){
            if(item.cast == 'N')
                is_cast = '<span class="red">(cast)</span>'
            
            if(item.cond == 'N')
                is_cond = '<span class="red">(cond)</span>'

            
            if(item.prod == 'N' && item.noprod == 'Y'){
                is_prod = '<span class="red">(prod)</span>'
            }
            
            if(item.location == 'N'){
                is_location = '<span class="red">(location)</span>'
            }
            
            
            if(item.info == 'N'){
                is_info = '<span class="red">(info)</span>'
            }
            
            
            if(item.nn == 'Y')
                is_nn = '<span class="red">(NN)</span>'
            
            if(item.role != '0'){
                if(item.role>1)
                    pluriel = 's';
                is_role = '<span class="red">(role'+pluriel+')</span>'
            }
            }
            items.push( '<div class="companies_work_bordered" id="'+item.id_prod+'"><div class="float_left '+class_type+'">'+prodtype+'</div><div class="float_left companies_work_period">'+item.period+'</div><div class="titre float_left" id="titre_'+item.id_prod+'"><span class="companies_prod_title" data-id_prod="'+item.id_prod+'" onclick="show_prod(\''+item.id_prod+'\',\''+statut+'\',\''+lang+'\')">'+item.title+'</span> <span class="footer_text_verticalbar">|</span> '+item.comp_name+' <span class="footer_text_verticalbar">|</span> '+item.venue+apply_admin+theatre_updt+euphonia+del+'</div><div class="cell">'+is_location+is_info+is_cond+is_prod+is_cast+is_nn+is_role+'</div><div class="clear_both"></div></div><div class="companies_prod_content" data-id="'+item.id_prod+'" id="content_'+item.id_prod+'"></div>' );

        });
        $( ".companies_result" ).html(items.join( "" )); 
        if(data=='')
            $( ".companies_result" ).html(''); 
        var widthMax = 0;
        $( ".companies_work_period" ).each(function(){
            var w = $(this).width();
            if(w>widthMax)
                widthMax=w;
        })
        $( ".companies_work_period" ).width(widthMax);
        
    })
                        
}
function open_all(){
    $('.prod_content').each(function(){
        var id = $(this).data('id')  
        $(this).slideToggle();
        if($('#content_'+id).html()==''){
            $('body').addClass("wait");
            setTimeout(function(){
                $.get( "companies/season_prod_fetch.php",{id : id}, function( data ) {
                    $('#content_'+id).html(data)
            $('body').addClass("default");     
                })
            },200)
        }
    })         
}
function del_updt(id,titre){
    var conf = confirm('Do you really want to delete the update and restaure '+titre);
    if(conf){
        $.post('companies/season_del_updt.php',{id : id},function( message ){
            alert(message)
            $.get( "companies/season_prod_fetch.php",{id : id}, function( data ) {
                $('#content_'+id).html(data)
                $("#page").fadeOut();
                $(".window").fadeOut();
            })
        })
    }       
}
function season_moins(season,code,ensid,statut,lang){
    var year = [];
    var total = $('#period option').length;
    $('#period option').each(function(index){
        year[index] = $(this).val();
    });    
    if($('#period').val()==year[total]){
        $('#season_plus').css('visibility','hidden');
    }
    $('#season_plus').css('visibility','visible');
    var yr =$('#period').val();           
    var before = $('#period option').filter('[value='+yr+']').prev();
    before = before.prop('value');
    
    $('.companies_refresh').remove();    
    fetch_seasons(season,code,ensid,before,statut,lang)
    
    $('#period option').removeAttr('selected').filter('[value='+before+']').prop('selected', true)  
    if($('#period').val()==year[0]){
        $('#season_moins').css('visibility','hidden');
    }       
}
function season_plus(season,code,ensid,statut,lang){
    var year = [];
    var total = $('#period option').length;
    $('#period option').each(function(index){
        year[index] = $(this).val();
    });    
    if($('#period').val()==year[total]){
        $('#season_plus').css('visibility','hidden');
    }
    $('#season_moins').css('visibility','visible');
    var yr =$('#period').val();     
    var next = $('#period option').filter('[value='+yr+']').next();
    next = next.prop('value');
    $('.companies_refresh').remove();   
    fetch_seasons(season,code,ensid,next,statut,lang) 
        
    $('#period option').removeAttr('selected').filter('[value='+next+']').prop('selected', true)  
    if($('#period').val()==year[total-1]){
        $('#season_plus').css('visibility','hidden');
    }
           
}
function set_month_break(season){
    var d = new Date();
    var year = d.getFullYear();
    var month = d.getMonth()+1;
    if(month.length==1)month = '0'+month;
    var month_break =  season.substring(0, 2);
        if(month>=month_break && month<=12){
$('#period option').removeAttr('selected').filter('[value='+year+']').attr('selected', true)
        }else{
$('#period option').removeAttr('selected').filter('[value='+(year-1)+']').attr('selected', true)
    }
}

function show_prod_perdate(id,statut,lang,line){
    $.get( "/ressources/views/festivals/season_prod_fetch.php",{id : id,statut : statut,lang : lang}, function( data ) {
        $('#content_'+id+'_'+line).html(data)
        $('#content_'+id+'_'+line).slideToggle(5)
        $('#content_'+id+'_'+line+' #dates_'+id).hide();
        $('.cities_oneprod_tourdate_title,.companies_prod_content_dotL_city,.cities_oneprod_dotE').hide();
    })
}
function company_info_close(){
    $('#company_info').fadeOut();
}
function addDays(date, days) {
    var dateFormatTotime = new Date(date);
        var increasedDate = new Date(dateFormatTotime.getTime() +(days *86400000));
        var month =  increasedDate.getMonth()+1;
        month = month.toString()
        if(month.length==1){
            month = '0'+month;
        }
        var day =  increasedDate.getDate();
        day = day.toString()
        if(day.length==1)
            day = '0'+day;
        var year = increasedDate.getFullYear();
        year =year.toString();
        return year+month+day;
}
function nearby_250(nb,lang,auth){    
        $('#back2comp').show();
        var default_venue = $('#default_venue').val();
        var city = $('#city_back').val();
        var from =  $('#from_back').val();
        var to =  $('#to_back').val();
        var company =  $('#company_back').val();
        var country =  $('#country_iso_back').val();
        var comp_id =  $('#comp_id_back').val();
        var work_id =  $('#work_id_back').val();
        $.post('/ressources/views/perfs/nearby250_'+nb+'.php',{work_id : work_id,comp_id: comp_id, country : country,city : city,default_venue : default_venue, from : from, to : to, company : company},function(data){
            $('#250_result').html(data);
            $('.companies_work_bordered,#cities_prod,.list_title_medium,#125_result,#no_calendar').hide();
            $('#250_result').fadeIn();
            
            $('.show_prod').click(function(){
                var ensid =  $(this).data('ens');
                var nb = $(this).data('nb');
                $.post('/ressources/views/perfs/comp_prog.php',{work_id : work_id,comp_id: comp_id,ensid : ensid, from : from, to : to, lang : lang, auth : auth},function(data){
                    (/alert(data))
                    
                    $('#prods'+nb).html(data);
                    $('#prods'+nb).slideToggle(); 
                })
            });
            $('.show_prod_city').click(function(){
                var city =  $(this).data('city');
                var country =  $(this).data('country');
                var nb = $(this).data('nb');
                $.post('/ressources/views/perfs/cities_prog.php',{work_id : work_id,comp_id: comp_id,city : city,country : country, from : from, to : to, lang : lang, auth : auth},function(data){
                    (/alert(data))
                    
                    $('#prods'+nb).html(data);
                    $('#prods'+nb).slideToggle(); 
                })
            });
        });
}
function nearby_125(nb,lang,auth){ 
        $('#back2comp').show();
        var default_venue = $('#default_venue').val();
        var city = $('#city_back').val();
        var from =  $('#from_back').val();
        var to =  $('#to_back').val();
        var country =  $('#country_iso_back').val();
        var company =  $('#company_back').val();
        var comp_id =  $('#comp_id_back').val();
        var work_id =  $('#work_id_back').val();
        $.post('/ressources/views/perfs/nearby125_'+nb+'.php',{work_id : work_id,comp_id: comp_id, country : country,city : city, default_venue : default_venue, from : from, to : to, company : company},function(data){
            $('#125_result').html(data);
            $('.companies_work_bordered,#cities_prod,.list_title_medium,#250_result,#no_calendar').hide();
            $('#125_result').fadeIn();
            $('.show_prod').click(function(){
                var ensid =  $(this).data('ens');
                var nb = $(this).data('nb');
                $.post('/ressources/views/perfs/comp_prog.php',{work_id : work_id,comp_id: comp_id,ensid : ensid, from : from, to : to, lang : lang, auth : auth},function(data){
                    (/alert(data))
                    
                    $('#prod'+nb).html(data);
                    $('#prod'+nb).slideToggle(); 
                })
            });
            $('.show_prod_city').click(function(){
                var city =  $(this).data('city');
                var country =  $(this).data('country');
                var nb = $(this).data('nb');
                $.post('/ressources/views/perfs/cities_prog.php',{city : city,country : country, from : from, to : to, lang : lang, auth : auth},function(data){
                    (/alert(data))
                    
                    $('#prod'+nb).html(data);
                    $('#prod'+nb).slideToggle(); 
                })
            });
        });
    
}

In [65]:
import easygui as eg
import sys

while 1:
    eg.msgbox("Hello, world!")

    msg ="What is your favorite flavor?"
    title = "Ice Cream Survey"
    choices = ["Vanilla", "Chocolate", "Strawberry", "Rocky Road"]
    choice = eg.choicebox(msg, title, choices)

    # note that we convert choice to string, in case
    # the user cancelled the choice, and we got None.
    eg.msgbox("You chose: " + str(choice), "Survey Result")

    msg = "Do you want to continue?"
    title = "Please Confirm"
    if eg.ccbox(msg, title):     # show a Continue/Cancel dialog
        pass  # user chose Continue
    else:
        sys.exit(0)           # user chose Canc

[0]


NameError: name 'sys' is not defined

In [10]:
from IPython.display import clear_output

for i in range(10):
    clear_output()
    print("Hello World!")

Hello World!


In [95]:
test = [[1,2,3],[4,5,6]]
print(test[[0,1]])


TypeError: list indices must be integers or slices, not list

In [48]:
len(temp)

39

In [140]:
test = 'div class="row"><div class="cities_oneprod_cell_prod"></div><div class="cell">~</div></div><div class="row"><div class="cities_oneprod_cell_role" id="role_74">Angelina</div><div class="cell cell_artist_name" id="artist_23479" style="padding-top: 0px"><a class="link_menu" href="/a/Dara Savinova/23479/en"  target="art"title=""  ><span  >Dara Savinova</span></a></div></div><div class="row"><div class="cities_oneprod_cell_role" id="role_69">Don Ramiro</div><div class="cell cell_artist_name" id="artist_12439" style="padding-top: 0px"><a class="link_menu" href="/a/Michele Angelini/12439/en"  target="art"title=""  ><span  >Michele Angelini</span></a></div></div><div class="row"><div class="cities_oneprod_cell_role" id="role_70">Dandini</div><div class="cell cell_artist_name" id="artist_8226" style="padding-top: 0px"><a class="link_menu" href="/a/Jens Persson/8226/en"  target="art"title=""  ><span  >Jens Persson</span></a></div></div><div class="row"><div class="cities_oneprod_cell_role" id="role_71">Don Magnifico</div><div class="cell cell_artist_name" id="artist_8548" style="padding-top: 0px"><a class="link_menu" href="/a/John Erik Eleby/8548/en"  target="art"title=""  ><span  >John Erik Eleby</span></a></div></div><div class="row"><div class="cities_oneprod_cell_role" id="role_72">Clorinda</div><div class="cell cell_artist_name" id="artist_74202" style="padding-top: 0px"><a class="link_menu" href="/a/Marianne Odencrants/74202/en"  target="art"title=""  ><span  >Marianne Odencrants</span></a></div></div><div class="row"><div class="cities_oneprod_cell_role" id="role_73">Tisbe</div><div class="cell cell_artist_name" id="artist_5650" style="padding-top: 0px"><a class="link_menu" href="/a/Francine Vis/5650/en"  target="art"title=""  ><span  >Francine Vis</span></a></div></div><div class="row"><div class="cities_oneprod_cell_role" id="role_75">Alidoro</div><div class="cell cell_artist_name" id="artist_11661" style="padding-top: 0px"><a class="link_menu" href="/a/Markus Schwartz/11661/en"  target="art"title=""  ><span  >Markus Schwartz</span></a></div></div><div class="row"></div></div><div class="cities_oneprod_dotE" id="dote_line"><img src="/ressources/pix/icons/theater.png" title="Programming company" alt="Programming company" style="margin-left:-1px; height: 25px; opacity: 0.5"/> Kungliga Operan</div></div>'
#re.findall('href="(.*?)"',test)

re.sub('href="(.*?)"', 'href="https://www.operabase.com\g<1>"',test)

'div class="row"><div class="cities_oneprod_cell_prod"></div><div class="cell">~</div></div><div class="row"><div class="cities_oneprod_cell_role" id="role_74">Angelina</div><div class="cell cell_artist_name" id="artist_23479" style="padding-top: 0px"><a class="link_menu" href="https://www.operabase.com/a/Dara Savinova/23479/en"  target="art"title=""  ><span  >Dara Savinova</span></a></div></div><div class="row"><div class="cities_oneprod_cell_role" id="role_69">Don Ramiro</div><div class="cell cell_artist_name" id="artist_12439" style="padding-top: 0px"><a class="link_menu" href="https://www.operabase.com/a/Michele Angelini/12439/en"  target="art"title=""  ><span  >Michele Angelini</span></a></div></div><div class="row"><div class="cities_oneprod_cell_role" id="role_70">Dandini</div><div class="cell cell_artist_name" id="artist_8226" style="padding-top: 0px"><a class="link_menu" href="https://www.operabase.com/a/Jens Persson/8226/en"  target="art"title=""  ><span  >Jens Persson</span>